In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, precision_score, accuracy_score, f1_score, recall_score
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Reading Padel Data :

In [ ]:
bitter_train_P = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Padel/Cleaned Dataset/bitterTrainCleaned.csv")
bitter_test_P = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Padel/Cleaned Dataset/bitterTestCleaned.csv")
sweet_train_P = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Padel/Cleaned Dataset/sweet_train_clean.csv")
sweet_test_P = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Padel/Cleaned Dataset/sweet_test_clean.csv")

Reading Mordred Data :

In [ ]:
bitter_train_M = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Mordred/Cleaned Data/bitter_train_v5.csv")
bitter_test_M = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Mordred/Cleaned Data/bitter_test_v5.csv")
sweet_train_M = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Mordred/Cleaned Data/sweet_train_v5.csv")
sweet_test_M = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Mordred/Cleaned Data/sweet_test_v5.csv")

Script :

In [ ]:
def findPredictions(x_Train, x_Test, y_Train, y_Test, data_type):

  #Preparing data for PCA
  pca2 = PCA(n_components=2)
  x_pca2_data_train = pca2.fit_transform(x_Train)
  x_pca2_data_test = pca2.fit_transform(x_Test)

  pca10 = PCA(n_components=10)
  x_pca10_data_train = pca10.fit_transform(x_Train)
  x_pca10_data_test = pca10.fit_transform(x_Test)

  
  
  #Preparing data for Correlation matrix on features
  train_data = pd.concat([x_Train,y_Train],axis=1)
  test_data = pd.concat([x_Test,y_Test],axis=1)
  train_corr , test_corr = correlation_check(train_data, test_data,0.7)

  if(data_type == 'Sweet'):
    x_Train_corr = train_corr.drop(['Sweet'],axis=1)
    x_Test_corr = test_corr.drop(['Sweet'],axis=1)

  if(data_type == 'Bitter'):
    x_Train_corr = train_corr.drop(['Bitter'],axis=1)
    x_Test_corr = test_corr.drop(['Bitter'],axis=1)

  
  
  #Preparing data for Correlation matrix on features and Labels

  a = x_Train_corr.corr(method ='pearson').abs()
  df_temp = a.head(1)
  df_temp.fillna(0,inplace = True)

  colList = []
  for i in df_temp.columns:
    if(df_temp[i].item() < 0.1):
      colList.append(i)

  for i in colList:
    new_x_train_corr = x_Train_corr.drop(i,axis = 1)

  for i in colList:
    new_x_test_corr = x_Test_corr.drop(i,axis = 1)


  

  #Preparing data for selectkBest
  from sklearn.feature_selection import SelectKBest, f_classif
  
  X_new = SelectKBest(f_classif, k=200).fit(x_Train, y_Train)
  cols = X_new.get_support(indices=True)
  features_df_new_train = x_Train.iloc[:,cols]
  features_df_new_test = x_Test.iloc[:,cols]

  



  #Applying Models : 
  #---------------------

  #1. Logistic Regression :
  from sklearn.linear_model import LogisticRegression

  clfLR1 = LogisticRegression(random_state=0)
  
  print("Logistic Regression : ")
  print("------------------------")
  print()

  
  clfLR_Basic = clfLR1.fit(x_Train, y_Train)
  y_pred_Basic = clfLR_Basic.predict(x_Test)
  print("Basic ==>                          Precision: ",precision_score(y_Test, y_pred_Basic)," Recall: ",recall_score(y_Test, y_pred_Basic),"  F1: ",f1_score(y_Test, y_pred_Basic, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_Basic))
  print()

  clfLR2 = LogisticRegression(random_state=0)

  clfLRpca2 = clfLR2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfLRpca2.predict(x_pca2_data_test)
  print("PCA with n = 2 ==>                 Precision: ",precision_score(y_Test, y_predPca2)," Recall: ",recall_score(y_Test, y_predPca2),"  F1: ",f1_score(y_Test, y_predPca2, average='binary')," Accuracy: ",accuracy_score(y_Test, y_predPca2))
  print()

  clfLR3 = LogisticRegression(random_state=0)

  clfLRpca10 = clfLR3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfLRpca10.predict(x_pca10_data_test)
  print("PCA with n = 10 ==>                Precision: ",precision_score(y_Test, y_predPca10)," Recall: ",recall_score(y_Test, y_predPca10),"  F1: ",f1_score(y_Test, y_predPca10, average='binary')," Accuracy: ",accuracy_score(y_Test, y_predPca10))
  print()

  clfLR4 = LogisticRegression(random_state=0)

  clfLR_CMF = clfLR4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfLR_CMF.predict(x_Test_corr)
  print("Correlation Matrix on Features ==> Precision: ",precision_score(y_Test, y_pred_CMF)," Recall: ",recall_score(y_Test, y_pred_CMF),"  F1: ",f1_score(y_Test, y_pred_CMF, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_CMF))
  print()

  clfLR5 = LogisticRegression(random_state=0)

  clfLR_CMFL = clfLR5.fit(new_x_train_corr, y_Train)
  y_pred_CMFL = clfLR_CMFL.predict(new_x_test_corr)
  print("CorMatrix on Features & Labels ==> Precision: ",precision_score(y_Test, y_pred_CMFL)," Recall: ",recall_score(y_Test, y_pred_CMFL),"  F1: ",f1_score(y_Test, y_pred_CMFL, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_CMFL))
  print()

  clfLR6 = LogisticRegression(random_state=0)

  clfLR_selectK = clfLR6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfLR_selectK.predict(features_df_new_test)
  print("SelectK best with k = 200 ==>      Precision: ",precision_score(y_Test, y_pred_selectK)," Recall: ",recall_score(y_Test, y_pred_selectK),"  F1: ",f1_score(y_Test, y_pred_selectK, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_selectK))
  print()

  #=====================================================================================================================================================

  #2. Random forgest :
  from sklearn.ensemble import RandomForestClassifier

  clfRF1 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=10, criterion='entropy', random_state=30)
  
  print("Random Forest : ")
  print("----------------")
  print()

  clfRF_Basic = clfRF1.fit(x_Train, y_Train)
  y_pred_Basic = clfRF_Basic.predict(x_Test)
  print("Basic ==>                          Precision: ",precision_score(y_Test, y_pred_Basic)," Recall: ",recall_score(y_Test, y_pred_Basic),"  F1: ",f1_score(y_Test, y_pred_Basic, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_Basic))
  print()

  clfRF2 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=10, criterion='entropy', random_state=30)


  clfRFpca2 = clfRF2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfRFpca2.predict(x_pca2_data_test)
  print("PCA with n = 2 ==>                 Precision: ",precision_score(y_Test, y_predPca2)," Recall: ",recall_score(y_Test, y_predPca2),"  F1: ",f1_score(y_Test, y_predPca2, average='binary')," Accuracy: ",accuracy_score(y_Test, y_predPca2))
  print()

  clfRF3 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=10, criterion='entropy', random_state=30)


  clfRFpca10 = clfRF3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfRFpca10.predict(x_pca10_data_test)
  print("PCA with n = 10 ==>                Precision: ",precision_score(y_Test, y_predPca10)," Recall: ",recall_score(y_Test, y_predPca10),"  F1: ",f1_score(y_Test, y_predPca10, average='binary')," Accuracy: ",accuracy_score(y_Test, y_predPca10))
  print()

  clfRF4 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=10, criterion='entropy', random_state=30)

  clfRF_CMF = clfRF4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfRF_CMF.predict(x_Test_corr)
  print("Correlation Matrix on Features ==> Precision: ",precision_score(y_Test, y_pred_CMF)," Recall: ",recall_score(y_Test, y_pred_CMF),"  F1: ",f1_score(y_Test, y_pred_CMF, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_CMF))
  print()

  clfRF5 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=10, criterion='entropy', random_state=30)

  clfLR_CMFL = clfRF5.fit(new_x_train_corr, y_Train)
  y_pred_CMFL = clfLR_CMFL.predict(new_x_test_corr)
  print("CorMatrix on Features & Labels ==> Precision: ",precision_score(y_Test, y_pred_CMFL)," Recall: ",recall_score(y_Test, y_pred_CMFL),"  F1: ",f1_score(y_Test, y_pred_CMFL, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_CMFL))
  print()


  clfRF6 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=10, criterion='entropy', random_state=30)

  clfRF_selectK = clfRF6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfRF_selectK.predict(features_df_new_test)
  print("SelectK best with k = 200 ==>      Precision: ",precision_score(y_Test, y_pred_selectK)," Recall: ",recall_score(y_Test, y_pred_selectK),"  F1: ",f1_score(y_Test, y_pred_selectK, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_selectK))
  print()


  


  #3. Adaboost
  from sklearn.ensemble import AdaBoostClassifier

  clfAB1 = AdaBoostClassifier(n_estimators=100, random_state=0)

  print("AdaBoost : ")
  print("----------------")
  print()

  clfAB_Basic = clfAB1.fit(x_Train, y_Train)
  y_pred_Basic = clfAB_Basic.predict(x_Test)
  print("Basic ==>                          Precision: ",precision_score(y_Test, y_pred_Basic)," Recall: ",recall_score(y_Test, y_pred_Basic),"  F1: ",f1_score(y_Test, y_pred_Basic, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_Basic))
  print()

  clfAB2 = AdaBoostClassifier(n_estimators=100, random_state=0)

  clfABpca2 = clfAB2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfABpca2.predict(x_pca2_data_test)
  print("PCA with n = 2 ==>                 Precision: ",precision_score(y_Test, y_predPca2)," Recall: ",recall_score(y_Test, y_predPca2),"  F1: ",f1_score(y_Test, y_predPca2, average='binary')," Accuracy: ",accuracy_score(y_Test, y_predPca2))
  print()

  clfAB3 = AdaBoostClassifier(n_estimators=100, random_state=0)

  clfABpca10 = clfAB3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfABpca10.predict(x_pca10_data_test)
  print("PCA with n = 10 ==>                Precision: ",precision_score(y_Test, y_predPca10)," Recall: ",recall_score(y_Test, y_predPca10),"  F1: ",f1_score(y_Test, y_predPca10, average='binary')," Accuracy: ",accuracy_score(y_Test, y_predPca10))
  print()

  clfAB4 = AdaBoostClassifier(n_estimators=100, random_state=0)

  clfAB_CMF = clfAB4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfAB_CMF.predict(x_Test_corr)
  print("Correlation Matrix on Features ==> Precision: ",precision_score(y_Test, y_pred_CMF)," Recall: ",recall_score(y_Test, y_pred_CMF),"  F1: ",f1_score(y_Test, y_pred_CMF, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_CMF))
  print()

  clfAB5 = AdaBoostClassifier(n_estimators=100, random_state=0)

  clfAB_CMFL = clfAB5.fit(new_x_train_corr, y_Train)
  y_pred_CMFL = clfAB_CMFL.predict(new_x_test_corr)
  print("CorMatrix on Features & Labels ==> Precision: ",precision_score(y_Test, y_pred_CMFL)," Recall: ",recall_score(y_Test, y_pred_CMFL),"  F1: ",f1_score(y_Test, y_pred_CMFL, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_CMFL))
  print()

  clfAB6 = AdaBoostClassifier(n_estimators=100, random_state=0)


  clfAB_selectK = clfAB6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfAB_selectK.predict(features_df_new_test)
  print("SelectK best with k = 200 ==>      Precision: ",precision_score(y_Test, y_pred_selectK)," Recall: ",recall_score(y_Test, y_pred_selectK),"  F1: ",f1_score(y_Test, y_pred_selectK, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_selectK))
  print()


  
  #4. XGBoostClassifier
  import xgboost as xg

  xgb_r1 = xg.XGBClassifier(max_depth=10)

  print("XGBoostClassifier : ")
  print("----------------")
  print()

  clfXGB_Basic = xgb_r1.fit(x_Train, y_Train)
  y_pred_Basic = clfXGB_Basic.predict(x_Test)
  print("Basic ==>                          Precision: ",precision_score(y_Test, y_pred_Basic)," Recall: ",recall_score(y_Test, y_pred_Basic),"  F1: ",f1_score(y_Test, y_pred_Basic, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_Basic))
  print()

  xgb_r2 = xg.XGBClassifier(max_depth=10)

  clfXGBpca2 = xgb_r2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfXGBpca2.predict(x_pca2_data_test)
  print("PCA with n = 2 ==>                 Precision: ",precision_score(y_Test, y_predPca2)," Recall: ",recall_score(y_Test, y_predPca2),"  F1: ",f1_score(y_Test, y_predPca2, average='binary')," Accuracy: ",accuracy_score(y_Test, y_predPca2))
  print()

  xgb_r3 = xg.XGBClassifier(max_depth=10)


  clfXGBpca10 = xgb_r3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfXGBpca10.predict(x_pca10_data_test)
  print("PCA with n = 10 ==>                Precision: ",precision_score(y_Test, y_predPca10)," Recall: ",recall_score(y_Test, y_predPca10),"  F1: ",f1_score(y_Test, y_predPca10, average='binary')," Accuracy: ",accuracy_score(y_Test, y_predPca10))
  print()

  xgb_r4 = xg.XGBClassifier(max_depth=10)

  clfXGB_CMF = xgb_r4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfXGB_CMF.predict(x_Test_corr)
  print("Correlation Matrix on Features ==> Precision: ",precision_score(y_Test, y_pred_CMF)," Recall: ",recall_score(y_Test, y_pred_CMF),"  F1: ",f1_score(y_Test, y_pred_CMF, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_CMF))
  print()

  xgb_r5 = xg.XGBClassifier(max_depth=10)

  clfXGB_CMFL = xgb_r5.fit(new_x_train_corr, y_Train)
  y_pred_CMFL = clfXGB_CMFL.predict(new_x_test_corr)
  print("CorMatrix on Features & Labels ==> Precision: ",precision_score(y_Test, y_pred_CMFL)," Recall: ",recall_score(y_Test, y_pred_CMFL),"  F1: ",f1_score(y_Test, y_pred_CMFL, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_CMFL))
  print()

  xgb_r6 = xg.XGBClassifier(max_depth=10)

  clfXGB_selectK = xgb_r6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfXGB_selectK.predict(features_df_new_test)
  print("SelectK best with k = 200 ==>      Precision: ",precision_score(y_Test, y_pred_selectK)," Recall: ",recall_score(y_Test, y_pred_selectK),"  F1: ",f1_score(y_Test, y_pred_selectK, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_selectK))
  print()

  
  

Supporting Functions :

In [ ]:
def correlation_check(traindata,testdata,thresh): # drop columns above certain threshold    
        corr_matrix = traindata.corr()
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
        to_drop = [column for column in upper.columns if any(upper[column] >thresh)]
        trainset=traindata.drop(traindata[to_drop], axis=1)
        testset=testdata.drop(testdata[to_drop],axis=1)
        return trainset,testset

In [ ]:
def gridSearchOnLogReg(x, y):
  grid={
        "C":np.logspace(-3,3,7,1,0),
        "penalty":["l1","l2","elasticnet","none"],
        "max_iter":[50,80,100],
        "solver":["newton-cg", "lbfgs", "liblinear", "sag", "saga"]
        }
  logreg=LogisticRegression()
  logreg_cv=GridSearchCV(logreg,grid)
  logreg_cv.fit(x,y)
  return logreg_cv

RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=8, criterion='entropy', random_state=30)

## **Padel :**

---



Sweet

In [ ]:
y_train = sweet_train_P['Sweet']
x_train = sweet_train_P.drop(['Sweet','Name','Name.1','Taste','Unnamed: 0'],axis=1)
y_train.replace({True:1, False:0},inplace=True)
y_test = sweet_test_P['Sweet']
x_test = sweet_test_P.drop(['Sweet','Name','Name.1','Taste','Unnamed: 0'],axis=1)
y_test.replace({True:1, False:0},inplace=True)
y_train = y_train.to_frame()
y_test = y_test.to_frame()

In [ ]:
findPredictions(x_train, x_test, y_train, y_test, "Sweet")

Logistic Regression : 
------------------------

Basic ==>                          Precision:  0.8382352941176471  Recall:  0.5277777777777778   F1:  0.6477272727272728  Accuracy:  0.6149068322981367

PCA with n = 2 ==>                 Precision:  0.6883116883116883  Recall:  0.9814814814814815   F1:  0.8091603053435115  Accuracy:  0.6894409937888198

PCA with n = 10 ==>                Precision:  0.6594202898550725  Recall:  0.8425925925925926   F1:  0.7398373983739838  Accuracy:  0.6024844720496895

Correlation Matrix on Features ==> Precision:  0.7363636363636363  Recall:  0.75   F1:  0.7431192660550459  Accuracy:  0.6521739130434783

CorMatrix on Features & Labels ==> Precision:  0.7297297297297297  Recall:  0.75   F1:  0.7397260273972601  Accuracy:  0.6459627329192547

SelectK best with k = 200 ==>      Precision:  0.92  Recall:  0.6388888888888888   F1:  0.7540983606557377  Accuracy:  0.7204968944099379

Random Forest : 
----------------

Basic ==>                          Preci

Bitter

In [ ]:
y_train = bitter_train_P['Bitter']
x_train = bitter_train_P.drop(['Bitter','Name','Name.1','Taste'],axis=1)
y_train.replace({True:1, False:0},inplace=True)
y_test = bitter_test_P['Bitter']
x_test = bitter_test_P.drop(['Bitter','Name','Name.1','Taste'],axis=1)
y_test.replace({True:1, False:0},inplace=True)
y_train = y_train.to_frame()
y_test = y_test.to_frame()

In [ ]:
findPredictions(x_train, x_test, y_train, y_test, "Bitter")

Logistic Regression : 
------------------------

Basic ==>                          Precision:  0.7407407407407407  Recall:  0.5714285714285714   F1:  0.6451612903225806  Accuracy:  0.6140350877192983

PCA with n = 2 ==>                 Precision:  0.14285714285714285  Recall:  0.009523809523809525   F1:  0.01785714285714286  Accuracy:  0.3567251461988304

PCA with n = 10 ==>                Precision:  0.375  Recall:  0.05714285714285714   F1:  0.09917355371900827  Accuracy:  0.36257309941520466

Correlation Matrix on Features ==> Precision:  0.8035714285714286  Recall:  0.42857142857142855   F1:  0.5590062111801243  Accuracy:  0.5847953216374269

CorMatrix on Features & Labels ==> Precision:  0.8  Recall:  0.41904761904761906   F1:  0.55  Accuracy:  0.5789473684210527

SelectK best with k = 200 ==>      Precision:  0.7083333333333334  Recall:  0.4857142857142857   F1:  0.576271186440678  Accuracy:  0.5614035087719298

Random Forest : 
----------------

Basic ==>                       

## **Mordred**

---



Sweet

In [ ]:
y_train = sweet_train_M['Sweet']
x_train = sweet_train_M.drop(['Sweet','Name','Taste'],axis=1)
y_train.replace({True:1, False:0},inplace=True)
y_test = sweet_test_M['Sweet']
x_test = sweet_test_M.drop(['Sweet','Name','Taste'],axis=1)
y_test.replace({True:1, False:0},inplace=True)
y_train = y_train.to_frame()
y_test = y_test.to_frame()

In [ ]:
findPredictions(x_train, x_test, y_train, y_test, "Sweet")

Logistic Regression : 
------------------------

Basic ==>                          Precision:  0.0  Recall:  0.0   F1:  0.0  Accuracy:  0.32679738562091504

PCA with n = 2 ==>                 Precision:  0.6644295302013423  Recall:  0.9611650485436893   F1:  0.7857142857142857  Accuracy:  0.6470588235294118

PCA with n = 10 ==>                Precision:  0.6644295302013423  Recall:  0.9611650485436893   F1:  0.7857142857142857  Accuracy:  0.6470588235294118

Correlation Matrix on Features ==> Precision:  0.0  Recall:  0.0   F1:  0.0  Accuracy:  0.32679738562091504

CorMatrix on Features & Labels ==> Precision:  0.0  Recall:  0.0   F1:  0.0  Accuracy:  0.32679738562091504

SelectK best with k = 200 ==>      Precision:  0.0  Recall:  0.0   F1:  0.0  Accuracy:  0.32679738562091504

Random Forest : 
----------------

Basic ==>                          Precision:  0.8589743589743589  Recall:  0.6504854368932039   F1:  0.7403314917127072  Accuracy:  0.6928104575163399

PCA with n = 2 ==>   

Bitter

In [ ]:
y_train = bitter_train_M['Bitter']
x_train = bitter_train_M.drop(['Bitter','Name','Taste'],axis=1)
y_train.replace({True:1, False:0},inplace=True)
y_test = bitter_test_M['Bitter']
x_test = bitter_test_M.drop(['Bitter','Name','Taste'],axis=1)
y_test.replace({True:1, False:0},inplace=True)
y_train = y_train.to_frame()
y_test = y_test.to_frame()

In [ ]:
findPredictions(x_train, x_test, y_train, y_test, "Bitter")

Logistic Regression : 
------------------------

Basic ==>                          Precision:  1.0  Recall:  0.009523809523809525   F1:  0.01886792452830189  Accuracy:  0.391812865497076

PCA with n = 2 ==>                 Precision:  0.4864864864864865  Recall:  0.5142857142857142   F1:  0.5  Accuracy:  0.3684210526315789

PCA with n = 10 ==>                Precision:  0.5877192982456141  Recall:  0.638095238095238   F1:  0.6118721461187215  Accuracy:  0.5029239766081871

Correlation Matrix on Features ==> Precision:  0.6140350877192983  Recall:  1.0   F1:  0.7608695652173912  Accuracy:  0.6140350877192983

CorMatrix on Features & Labels ==> Precision:  0.6140350877192983  Recall:  1.0   F1:  0.7608695652173912  Accuracy:  0.6140350877192983

SelectK best with k = 200 ==>      Precision:  0.6140350877192983  Recall:  1.0   F1:  0.7608695652173912  Accuracy:  0.6140350877192983

Random Forest : 
----------------

Basic ==>                          Precision:  0.8404255319148937  Recal

**Boruta**

In [ ]:
pip install Boruta

In [ ]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=8)
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1, max_iter=200)

In [ ]:
y_train = sweet_train_M['Sweet']
x_train = sweet_train_M.drop(['Sweet','Name','Taste'],axis=1)

y_train = y_train.to_frame()

In [ ]:
x = x_train.iloc[:,:].values
y = y_train.iloc[:,:].values

In [ ]:
y = y.ravel()

In [ ]:
feat_selector.fit(x, y)

Iteration: 	1 / 200
Confirmed: 	0
Tentative: 	1613
Rejected: 	0
Iteration: 	2 / 200
Confirmed: 	0
Tentative: 	1613
Rejected: 	0
Iteration: 	3 / 200
Confirmed: 	0
Tentative: 	1613
Rejected: 	0
Iteration: 	4 / 200
Confirmed: 	0
Tentative: 	1613
Rejected: 	0
Iteration: 	5 / 200
Confirmed: 	0
Tentative: 	1613
Rejected: 	0
Iteration: 	6 / 200
Confirmed: 	0
Tentative: 	1613
Rejected: 	0
Iteration: 	7 / 200
Confirmed: 	0
Tentative: 	1613
Rejected: 	0
Iteration: 	8 / 200
Confirmed: 	208
Tentative: 	438
Rejected: 	967
Iteration: 	9 / 200
Confirmed: 	208
Tentative: 	438
Rejected: 	967
Iteration: 	10 / 200
Confirmed: 	208
Tentative: 	438
Rejected: 	967
Iteration: 	11 / 200
Confirmed: 	208
Tentative: 	438
Rejected: 	967
Iteration: 	12 / 200
Confirmed: 	229
Tentative: 	321
Rejected: 	1063
Iteration: 	13 / 200
Confirmed: 	229
Tentative: 	321
Rejected: 	1063
Iteration: 	14 / 200
Confirmed: 	229
Tentative: 	321
Rejected: 	1063
Iteration: 	15 / 200
Confirmed: 	229
Tentative: 	321
Rejected: 	1063
Iterat

BorutaPy(estimator=RandomForestClassifier(class_weight='balanced', max_depth=8,
                                          n_estimators=304, n_jobs=-1,
                                          random_state=RandomState(MT19937) at 0x7F1CAD158E20),
         max_iter=200, n_estimators='auto',
         random_state=RandomState(MT19937) at 0x7F1CAD158E20, verbose=2)

In [ ]:
listA = []
for i in range(0, len(feat_selector.support_)):
  if  feat_selector.support_[i]== False:
    listA.append(i)
    

In [ ]:
len(listA)

1371

In [ ]:
x_new = x_train.drop(x_train.columns[listA],
                       axis = 1)

In [ ]:
x_new

,nN,ATS2dv,ATS3s,ATS4s,ATS0Z,ATS0m,ATS3m,ATS0p,AATS0d,AATS3d,AATS4d,AATS0Z,AATS2Z,AATS0m,AATS1m,AATS0v,AATS1v,AATS2v,AATS3v,AATS4v,AATS0p,AATS1p,AATS2p,AATS3p,AATS4p,AATS0i,AATS1i,AATS3i,AATS4i,AATS5i,AATS7i,ATSC2c,ATSC3c,ATSC4c,ATSC5c,ATSC1d,ATSC0s,ATSC6Z,ATSC6m,ATSC6v,...,AMID_N,MID_O,AMID_O,MPC2,MPC3,MPC4,MPC5,MPC6,MPC7,MPC8,MPC9,TMPC10,piPC1,piPC2,piPC3,piPC4,piPC5,piPC6,piPC7,piPC8,piPC9,piPC10,TpiPC10,nRing,JGI3,MWC03,MWC04,MWC05,MWC06,MWC07,MWC08,MWC09,MWC10,TMWC10,SRW06,SRW08,SRW10,AMW,WPol,Zagreb2
0,0,532.000000,652.500000,629.083333,1158.0,4569.170871,9413.865359,50.323528,3.155556,2.830645,2.541353,25.733333,23.192771,101.537130,96.335918,181.029861,233.387037,201.265826,160.826674,144.999343,1.118301,1.445816,1.258039,1.017477,0.952669,169.548550,153.042175,166.058828,167.454386,169.891248,168.161237,-0.652482,1.204605,-0.670753,0.228861,12.975309,160.554938,-5.453827,-19.969966,125.602063,...,0.000000,20.160229,0.876532,36,51,62,73,74,70,70,68,608,3.218876,3.610918,3.951244,4.143135,4.304065,4.317488,4.262680,4.262680,4.234107,4.060443,6.411818,2,0.082526,5.686975,6.602588,7.513709,8.434464,9.350189,10.272738,11.190528,12.113908,122.960890,6.849066,8.525360,10.247042,7.602582,43,147.0
1,0,477.111111,509.601852,424.106996,1830.0,7568.326176,11909.663100,61.317277,3.309524,2.895652,2.603306,43.571429,27.100000,180.198242,118.231375,212.281739,255.059582,215.170710,179.685807,165.227306,1.459935,1.669930,1.435949,1.191106,1.173424,167.215625,150.288731,164.818253,165.667994,168.181859,166.072550,-0.461902,0.767067,-0.461966,0.177657,10.282880,111.512642,-8.217687,-35.280625,-9.450500,...,0.000000,14.925656,0.648942,36,51,62,73,74,70,70,68,608,3.218876,3.610918,3.951244,4.143135,4.304065,4.317488,4.262680,4.262680,4.234107,4.060443,6.411818,2,0.082526,5.686975,6.602588,7.513709,8.434464,9.350189,10.272738,11.190528,12.113908,122.960890,6.849066,8.525360,10.247042,9.428918,43,147.0
2,2,348.000000,327.027778,372.666667,940.0,3710.218949,5232.341659,52.683652,2.923077,2.697674,2.467391,24.102564,20.676923,95.133819,92.739769,206.598109,251.559891,206.377537,173.668198,157.736959,1.350863,1.675350,1.429224,1.235235,1.107106,165.471161,150.112578,161.777305,165.699596,164.954150,163.395314,0.145983,0.338385,-0.698717,0.537398,4.692308,117.287749,66.000000,316.956515,387.214257,...,0.178784,8.731863,0.415803,27,30,33,37,32,31,29,24,301,3.332205,3.749504,4.020877,4.312476,4.722398,4.452165,4.599529,4.819273,4.884694,4.627421,6.767271,1,0.043981,5.351858,6.175867,6.978214,7.805882,8.618485,9.447229,10.264966,11.093995,112.311208,6.447306,7.951207,9.493487,7.541579,27,105.0
3,0,254.000000,393.500000,344.166667,612.0,2413.585500,4575.372616,25.927979,3.000000,2.460317,1.772727,25.500000,23.047619,100.566063,92.993879,175.518792,227.079812,198.191463,153.270562,114.014016,1.080332,1.407855,1.237989,0.975734,0.796130,170.520236,154.381429,166.741525,172.134683,178.032893,184.993831,-0.307929,0.683571,-0.379936,-0.025630,6.000000,95.268229,37.000000,176.254520,243.836186,...,0.000000,10.677995,0.889833,18,23,22,23,17,7,1,0,135,2.564949,2.944439,3.178054,3.135494,3.178054,2.890372,2.079442,0.693147,0.000000,0.000000,4.912655,1,0.065625,4.962845,5.849325,6.722630,7.610358,8.487764,9.375092,10.253827,11.140687,92.513402,6.163315,7.845024,9.569063,7.502641,20,71.0
4,0,518.000000,682.541667,614.847222,1158.0,4569.170871,9127.853377,50.323528,3.155556,2.661290,2.270677,25.733333,23.192771,101.537130,96.335918,181.029861,233.387037,201.265826,157.905783,132.343321,1.118301,1.445816,1.258039,1.008267,0.897097,169.548550,153.042175,166.015110,169.040621,171.734788,166.939623,-0.628096,1.224730,-0.715289,0.159067,10.975309,160.554938,87.169383,415.124614,512.973197,...,0.000000,20.110824,0.874384,36,49,56,62,58,53,58,67,557,3.218876,3.610918,3.912023,4.043051,4.143135,4.077537,3.988984,4.077537,4.219508,4.276666,6.324359,2,0.077405,5.673323,6.575076,7.467942,8.371242,9.267193,10.170725,11.068184,11.971710,122.361186,6.836259,8.493105,10.192

In [ ]:
y_test = sweet_test_M['Sweet']
x_test = sweet_test_M.drop(['Sweet','Name','Taste'],axis=1)

In [ ]:
y_test = y_test.to_frame()

In [ ]:
x_new_test = x_test.drop(x_test.columns[listA],
                       axis = 1)

In [ ]:
x_new_test

,nN,ATS2dv,ATS3s,ATS4s,ATS0Z,ATS0m,ATS3m,ATS0p,AATS0d,AATS3d,AATS4d,AATS0Z,AATS2Z,AATS0m,AATS1m,AATS0v,AATS1v,AATS2v,AATS3v,AATS4v,AATS0p,AATS1p,AATS2p,AATS3p,AATS4p,AATS0i,AATS1i,AATS3i,AATS4i,AATS5i,AATS7i,ATSC2c,ATSC3c,ATSC4c,ATSC5c,ATSC1d,ATSC0s,ATSC6Z,ATSC6m,ATSC6v,...,AMID_N,MID_O,AMID_O,MPC2,MPC3,MPC4,MPC5,MPC6,MPC7,MPC8,MPC9,TMPC10,piPC1,piPC2,piPC3,piPC4,piPC5,piPC6,piPC7,piPC8,piPC9,piPC10,TpiPC10,nRing,JGI3,MWC03,MWC04,MWC05,MWC06,MWC07,MWC08,MWC09,MWC10,TMWC10,SRW06,SRW08,SRW10,AMW,WPol,Zagreb2
0,3,264.666667,321.879630,375.703704,1268.0,4982.059799,6262.950338,73.907442,2.614035,2.227941,2.012121,22.245614,18.788462,87.404558,83.558362,175.338892,223.299893,175.993860,149.859311,116.140023,1.296622,1.618120,1.354649,1.239443,0.938069,168.663989,151.396198,160.057177,171.281214,165.738477,163.361191,0.159971,-0.061525,-0.154919,0.311617,9.894737,77.072759,314.066482,1492.778791,2251.021667,...,0.239219,7.359986,0.306666,34,36,42,27,26,27,29,25,318,3.258097,3.610918,3.637586,3.806662,3.367296,3.332205,3.367296,3.433987,3.332205,3.258097,5.805135,1,0.113426,5.549076,6.444131,7.257708,8.172164,9.003685,9.935035,10.780060,11.725590,121.629623,6.829794,8.509363,10.256922,6.372266,27,128.0
1,0,434.000000,654.250000,643.888889,1160.0,4571.202999,8589.759775,51.212754,2.851064,2.603175,2.362319,24.680851,21.144578,97.259638,90.798347,174.649175,226.166478,191.396631,152.142943,138.329306,1.089633,1.421628,1.231107,0.994737,0.924393,170.202555,153.721352,166.302447,168.816659,168.959562,169.501042,-0.662971,1.107631,-0.544458,0.071001,10.191489,176.053191,-24.065188,-117.103836,-213.618083,...,0.000000,19.739869,0.858255,32,42,46,53,54,50,46,39,433,3.178054,3.496508,3.761200,3.850148,3.988984,4.007333,3.931826,3.850148,3.688879,3.258097,6.073045,1,0.059295,5.556828,6.424869,7.295056,8.170469,9.046762,9.925200,10.803954,11.683857,119.616526,6.688355,8.303752,9.968198,7.321955,39,129.0
2,0,428.000000,676.361111,663.638889,1160.0,4571.202999,8468.693766,51.212754,2.851064,2.492063,2.137681,24.680851,21.144578,97.259638,90.798347,174.649175,226.166478,191.396631,150.356217,129.443088,1.089633,1.421628,1.231107,0.986749,0.878757,170.202555,153.721352,166.259059,170.120488,171.211079,170.466790,-0.662283,1.126649,-0.537403,0.032920,9.191489,176.053191,15.739701,73.310736,47.808010,...,0.000000,19.723213,0.857531,32,41,42,45,43,38,36,36,392,3.178054,3.496508,3.737670,3.761200,3.828641,3.784190,3.663562,3.610918,3.610918,3.526361,5.973810,1,0.060855,5.549076,6.405228,7.261927,8.122371,8.983314,9.846017,10.709137,11.573409,119.160011,6.680855,8.283747,9.932901,7.321955,38,128.0
3,0,92.000000,219.916667,185.666667,410.0,1611.089128,1966.301776,18.174545,2.222222,1.743590,1.333333,22.777778,17.214286,89.504952,78.741107,159.470453,205.761349,164.089267,119.448125,92.036166,1.009697,1.326141,1.109793,0.836084,0.684056,172.119949,156.072281,170.501551,176.216465,182.379170,184.917652,-0.219335,0.378624,-0.175020,-0.012729,1.888889,74.641975,19.555556,93.749126,132.198175,...,0.000000,6.798694,0.849837,8,8,4,1,0,0,0,0,36,2.079442,2.197225,2.197225,1.609438,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,3.610918,0,0.046875,4.143135,4.875197,5.609472,6.347389,7.085901,7.825245,8.564649,9.304286,72.189262,5.209486,6.642487,8.103192,6.780995,8,31.0
4,0,1189.000000,1302.611111,1463.708333,2580.0,10150.424456,24297.097283,144.232646,3.293103,3.174263,2.707368,22.241379,20.792373,87.503659,89.054234,188.393973,244.945856,211.245372,178.576482,150.884097,1.243385,1.627610,1.433857,1.251932,1.107180,165.960067,147.856207,157.989833,161.973644,164.175074,162.077837,-0.793969,1.616018,-0.948455,0.252029,37.854935,284.355364,478.282996,2270.460414,2757.360945,...,0.000000,33.463807,0.597568,99,146,199,271,347,431,544,691,3670,4.174387,4.644391,5.036953,5.342334,5.659482,5.908083,6.146329,6.378426,6.626718,6.797940,8.279190,7,0.065662,6.700731,7.653969,8.595080,9.554852,10.503395,11.468294,12.421914,13.391032,204.066919,7.893945,9.631219,11.42

In [ ]:
x_new

,nN,ATS2dv,ATS3s,ATS4s,ATS0Z,ATS0m,ATS3m,ATS0p,AATS0d,AATS3d,AATS4d,AATS0Z,AATS2Z,AATS0m,AATS1m,AATS0v,AATS1v,AATS2v,AATS3v,AATS4v,AATS0p,AATS1p,AATS2p,AATS3p,AATS4p,AATS0i,AATS1i,AATS3i,AATS4i,AATS5i,AATS7i,ATSC2c,ATSC3c,ATSC4c,ATSC5c,ATSC1d,ATSC0s,ATSC6Z,ATSC6m,ATSC6v,...,AMID_N,MID_O,AMID_O,MPC2,MPC3,MPC4,MPC5,MPC6,MPC7,MPC8,MPC9,TMPC10,piPC1,piPC2,piPC3,piPC4,piPC5,piPC6,piPC7,piPC8,piPC9,piPC10,TpiPC10,nRing,JGI3,MWC03,MWC04,MWC05,MWC06,MWC07,MWC08,MWC09,MWC10,TMWC10,SRW06,SRW08,SRW10,AMW,WPol,Zagreb2
0,0,532.000000,652.500000,629.083333,1158.0,4569.170871,9413.865359,50.323528,3.155556,2.830645,2.541353,25.733333,23.192771,101.537130,96.335918,181.029861,233.387037,201.265826,160.826674,144.999343,1.118301,1.445816,1.258039,1.017477,0.952669,169.548550,153.042175,166.058828,167.454386,169.891248,168.161237,-0.652482,1.204605,-0.670753,0.228861,12.975309,160.554938,-5.453827,-19.969966,125.602063,...,0.000000,20.160229,0.876532,36,51,62,73,74,70,70,68,608,3.218876,3.610918,3.951244,4.143135,4.304065,4.317488,4.262680,4.262680,4.234107,4.060443,6.411818,2,0.082526,5.686975,6.602588,7.513709,8.434464,9.350189,10.272738,11.190528,12.113908,122.960890,6.849066,8.525360,10.247042,7.602582,43,147.0
1,0,477.111111,509.601852,424.106996,1830.0,7568.326176,11909.663100,61.317277,3.309524,2.895652,2.603306,43.571429,27.100000,180.198242,118.231375,212.281739,255.059582,215.170710,179.685807,165.227306,1.459935,1.669930,1.435949,1.191106,1.173424,167.215625,150.288731,164.818253,165.667994,168.181859,166.072550,-0.461902,0.767067,-0.461966,0.177657,10.282880,111.512642,-8.217687,-35.280625,-9.450500,...,0.000000,14.925656,0.648942,36,51,62,73,74,70,70,68,608,3.218876,3.610918,3.951244,4.143135,4.304065,4.317488,4.262680,4.262680,4.234107,4.060443,6.411818,2,0.082526,5.686975,6.602588,7.513709,8.434464,9.350189,10.272738,11.190528,12.113908,122.960890,6.849066,8.525360,10.247042,9.428918,43,147.0
2,2,348.000000,327.027778,372.666667,940.0,3710.218949,5232.341659,52.683652,2.923077,2.697674,2.467391,24.102564,20.676923,95.133819,92.739769,206.598109,251.559891,206.377537,173.668198,157.736959,1.350863,1.675350,1.429224,1.235235,1.107106,165.471161,150.112578,161.777305,165.699596,164.954150,163.395314,0.145983,0.338385,-0.698717,0.537398,4.692308,117.287749,66.000000,316.956515,387.214257,...,0.178784,8.731863,0.415803,27,30,33,37,32,31,29,24,301,3.332205,3.749504,4.020877,4.312476,4.722398,4.452165,4.599529,4.819273,4.884694,4.627421,6.767271,1,0.043981,5.351858,6.175867,6.978214,7.805882,8.618485,9.447229,10.264966,11.093995,112.311208,6.447306,7.951207,9.493487,7.541579,27,105.0
3,0,254.000000,393.500000,344.166667,612.0,2413.585500,4575.372616,25.927979,3.000000,2.460317,1.772727,25.500000,23.047619,100.566063,92.993879,175.518792,227.079812,198.191463,153.270562,114.014016,1.080332,1.407855,1.237989,0.975734,0.796130,170.520236,154.381429,166.741525,172.134683,178.032893,184.993831,-0.307929,0.683571,-0.379936,-0.025630,6.000000,95.268229,37.000000,176.254520,243.836186,...,0.000000,10.677995,0.889833,18,23,22,23,17,7,1,0,135,2.564949,2.944439,3.178054,3.135494,3.178054,2.890372,2.079442,0.693147,0.000000,0.000000,4.912655,1,0.065625,4.962845,5.849325,6.722630,7.610358,8.487764,9.375092,10.253827,11.140687,92.513402,6.163315,7.845024,9.569063,7.502641,20,71.0
4,0,518.000000,682.541667,614.847222,1158.0,4569.170871,9127.853377,50.323528,3.155556,2.661290,2.270677,25.733333,23.192771,101.537130,96.335918,181.029861,233.387037,201.265826,157.905783,132.343321,1.118301,1.445816,1.258039,1.008267,0.897097,169.548550,153.042175,166.015110,169.040621,171.734788,166.939623,-0.628096,1.224730,-0.715289,0.159067,10.975309,160.554938,87.169383,415.124614,512.973197,...,0.000000,20.110824,0.874384,36,49,56,62,58,53,58,67,557,3.218876,3.610918,3.912023,4.043051,4.143135,4.077537,3.988984,4.077537,4.219508,4.276666,6.324359,2,0.077405,5.673323,6.575076,7.467942,8.371242,9.267193,10.170725,11.068184,11.971710,122.361186,6.836259,8.493105,10.192

In [ ]:
y_train

,Sweet
0,True
1,True
2,True
3,True
4,True
...,...
2162,False
2163,False
2164,False
2165,False


In [ ]:
df_test = x_new_test.join(y_test)

In [ ]:
df_test

,nN,ATS2dv,ATS3s,ATS4s,ATS0Z,ATS0m,ATS3m,ATS0p,AATS0d,AATS3d,AATS4d,AATS0Z,AATS2Z,AATS0m,AATS1m,AATS0v,AATS1v,AATS2v,AATS3v,AATS4v,AATS0p,AATS1p,AATS2p,AATS3p,AATS4p,AATS0i,AATS1i,AATS3i,AATS4i,AATS5i,AATS7i,ATSC2c,ATSC3c,ATSC4c,ATSC5c,ATSC1d,ATSC0s,ATSC6Z,ATSC6m,ATSC6v,...,MID_O,AMID_O,MPC2,MPC3,MPC4,MPC5,MPC6,MPC7,MPC8,MPC9,TMPC10,piPC1,piPC2,piPC3,piPC4,piPC5,piPC6,piPC7,piPC8,piPC9,piPC10,TpiPC10,nRing,JGI3,MWC03,MWC04,MWC05,MWC06,MWC07,MWC08,MWC09,MWC10,TMWC10,SRW06,SRW08,SRW10,AMW,WPol,Zagreb2,Sweet
0,3,264.666667,321.879630,375.703704,1268.0,4982.059799,6262.950338,73.907442,2.614035,2.227941,2.012121,22.245614,18.788462,87.404558,83.558362,175.338892,223.299893,175.993860,149.859311,116.140023,1.296622,1.618120,1.354649,1.239443,0.938069,168.663989,151.396198,160.057177,171.281214,165.738477,163.361191,0.159971,-0.061525,-0.154919,0.311617,9.894737,77.072759,314.066482,1492.778791,2251.021667,...,7.359986,0.306666,34,36,42,27,26,27,29,25,318,3.258097,3.610918,3.637586,3.806662,3.367296,3.332205,3.367296,3.433987,3.332205,3.258097,5.805135,1,0.113426,5.549076,6.444131,7.257708,8.172164,9.003685,9.935035,10.780060,11.725590,121.629623,6.829794,8.509363,10.256922,6.372266,27,128.0,True
1,0,434.000000,654.250000,643.888889,1160.0,4571.202999,8589.759775,51.212754,2.851064,2.603175,2.362319,24.680851,21.144578,97.259638,90.798347,174.649175,226.166478,191.396631,152.142943,138.329306,1.089633,1.421628,1.231107,0.994737,0.924393,170.202555,153.721352,166.302447,168.816659,168.959562,169.501042,-0.662971,1.107631,-0.544458,0.071001,10.191489,176.053191,-24.065188,-117.103836,-213.618083,...,19.739869,0.858255,32,42,46,53,54,50,46,39,433,3.178054,3.496508,3.761200,3.850148,3.988984,4.007333,3.931826,3.850148,3.688879,3.258097,6.073045,1,0.059295,5.556828,6.424869,7.295056,8.170469,9.046762,9.925200,10.803954,11.683857,119.616526,6.688355,8.303752,9.968198,7.321955,39,129.0,True
2,0,428.000000,676.361111,663.638889,1160.0,4571.202999,8468.693766,51.212754,2.851064,2.492063,2.137681,24.680851,21.144578,97.259638,90.798347,174.649175,226.166478,191.396631,150.356217,129.443088,1.089633,1.421628,1.231107,0.986749,0.878757,170.202555,153.721352,166.259059,170.120488,171.211079,170.466790,-0.662283,1.126649,-0.537403,0.032920,9.191489,176.053191,15.739701,73.310736,47.808010,...,19.723213,0.857531,32,41,42,45,43,38,36,36,392,3.178054,3.496508,3.737670,3.761200,3.828641,3.784190,3.663562,3.610918,3.610918,3.526361,5.973810,1,0.060855,5.549076,6.405228,7.261927,8.122371,8.983314,9.846017,10.709137,11.573409,119.160011,6.680855,8.283747,9.932901,7.321955,38,128.0,True
3,0,92.000000,219.916667,185.666667,410.0,1611.089128,1966.301776,18.174545,2.222222,1.743590,1.333333,22.777778,17.214286,89.504952,78.741107,159.470453,205.761349,164.089267,119.448125,92.036166,1.009697,1.326141,1.109793,0.836084,0.684056,172.119949,156.072281,170.501551,176.216465,182.379170,184.917652,-0.219335,0.378624,-0.175020,-0.012729,1.888889,74.641975,19.555556,93.749126,132.198175,...,6.798694,0.849837,8,8,4,1,0,0,0,0,36,2.079442,2.197225,2.197225,1.609438,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,3.610918,0,0.046875,4.143135,4.875197,5.609472,6.347389,7.085901,7.825245,8.564649,9.304286,72.189262,5.209486,6.642487,8.103192,6.780995,8,31.0,True
4,0,1189.000000,1302.611111,1463.708333,2580.0,10150.424456,24297.097283,144.232646,3.293103,3.174263,2.707368,22.241379,20.792373,87.503659,89.054234,188.393973,244.945856,211.245372,178.576482,150.884097,1.243385,1.627610,1.433857,1.251932,1.107180,165.960067,147.856207,157.989833,161.973644,164.175074,162.077837,-0.793969,1.616018,-0.948455,0.252029,37.854935,284.355364,478.282996,2270.460414,2757.360945,...,33.463807,0.597568,99,146,199,271,347,431,544,691,3670,4.174387,4.644391,5.036953,5.342334,5.659482,5.908083,6.146329,6.378426,6.626718,6.797940,8.279190,7,0.065662,6.700731,7.653969,8.595080,9.554852,10.503395,11.468294,12.421914,13.391032,204.066919,7.893945,9.631219,11.421851,6.934293,123,406.0,Tr

In [ ]:
df_test = df_test.drop(df_test.columns[0],
                       axis = 1)

In [ ]:
df_test

,ATS2dv,ATS3s,ATS4s,ATS0Z,ATS0m,ATS3m,ATS0p,AATS0d,AATS3d,AATS4d,AATS0Z,AATS2Z,AATS0m,AATS1m,AATS0v,AATS1v,AATS2v,AATS3v,AATS4v,AATS0p,AATS1p,AATS2p,AATS3p,AATS4p,AATS0i,AATS1i,AATS3i,AATS4i,AATS5i,AATS7i,ATSC2c,ATSC3c,ATSC4c,ATSC5c,ATSC1d,ATSC0s,ATSC6Z,ATSC6m,ATSC6v,ATSC3se,...,MID_O,AMID_O,MPC2,MPC3,MPC4,MPC5,MPC6,MPC7,MPC8,MPC9,TMPC10,piPC1,piPC2,piPC3,piPC4,piPC5,piPC6,piPC7,piPC8,piPC9,piPC10,TpiPC10,nRing,JGI3,MWC03,MWC04,MWC05,MWC06,MWC07,MWC08,MWC09,MWC10,TMWC10,SRW06,SRW08,SRW10,AMW,WPol,Zagreb2,Sweet
0,264.666667,321.879630,375.703704,1268.0,4982.059799,6262.950338,73.907442,2.614035,2.227941,2.012121,22.245614,18.788462,87.404558,83.558362,175.338892,223.299893,175.993860,149.859311,116.140023,1.296622,1.618120,1.354649,1.239443,0.938069,168.663989,151.396198,160.057177,171.281214,165.738477,163.361191,0.159971,-0.061525,-0.154919,0.311617,9.894737,77.072759,314.066482,1492.778791,2251.021667,-1.003921,...,7.359986,0.306666,34,36,42,27,26,27,29,25,318,3.258097,3.610918,3.637586,3.806662,3.367296,3.332205,3.367296,3.433987,3.332205,3.258097,5.805135,1,0.113426,5.549076,6.444131,7.257708,8.172164,9.003685,9.935035,10.780060,11.725590,121.629623,6.829794,8.509363,10.256922,6.372266,27,128.0,True
1,434.000000,654.250000,643.888889,1160.0,4571.202999,8589.759775,51.212754,2.851064,2.603175,2.362319,24.680851,21.144578,97.259638,90.798347,174.649175,226.166478,191.396631,152.142943,138.329306,1.089633,1.421628,1.231107,0.994737,0.924393,170.202555,153.721352,166.302447,168.816659,168.959562,169.501042,-0.662971,1.107631,-0.544458,0.071001,10.191489,176.053191,-24.065188,-117.103836,-213.618083,3.093624,...,19.739869,0.858255,32,42,46,53,54,50,46,39,433,3.178054,3.496508,3.761200,3.850148,3.988984,4.007333,3.931826,3.850148,3.688879,3.258097,6.073045,1,0.059295,5.556828,6.424869,7.295056,8.170469,9.046762,9.925200,10.803954,11.683857,119.616526,6.688355,8.303752,9.968198,7.321955,39,129.0,True
2,428.000000,676.361111,663.638889,1160.0,4571.202999,8468.693766,51.212754,2.851064,2.492063,2.137681,24.680851,21.144578,97.259638,90.798347,174.649175,226.166478,191.396631,150.356217,129.443088,1.089633,1.421628,1.231107,0.986749,0.878757,170.202555,153.721352,166.259059,170.120488,171.211079,170.466790,-0.662283,1.126649,-0.537403,0.032920,9.191489,176.053191,15.739701,73.310736,47.808010,3.069908,...,19.723213,0.857531,32,41,42,45,43,38,36,36,392,3.178054,3.496508,3.737670,3.761200,3.828641,3.784190,3.663562,3.610918,3.610918,3.526361,5.973810,1,0.060855,5.549076,6.405228,7.261927,8.122371,8.983314,9.846017,10.709137,11.573409,119.160011,6.680855,8.283747,9.932901,7.321955,38,128.0,True
3,92.000000,219.916667,185.666667,410.0,1611.089128,1966.301776,18.174545,2.222222,1.743590,1.333333,22.777778,17.214286,89.504952,78.741107,159.470453,205.761349,164.089267,119.448125,92.036166,1.009697,1.326141,1.109793,0.836084,0.684056,172.119949,156.072281,170.501551,176.216465,182.379170,184.917652,-0.219335,0.378624,-0.175020,-0.012729,1.888889,74.641975,19.555556,93.749126,132.198175,0.994598,...,6.798694,0.849837,8,8,4,1,0,0,0,0,36,2.079442,2.197225,2.197225,1.609438,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,3.610918,0,0.046875,4.143135,4.875197,5.609472,6.347389,7.085901,7.825245,8.564649,9.304286,72.189262,5.209486,6.642487,8.103192,6.780995,8,31.0,True
4,1189.000000,1302.611111,1463.708333,2580.0,10150.424456,24297.097283,144.232646,3.293103,3.174263,2.707368,22.241379,20.792373,87.503659,89.054234,188.393973,244.945856,211.245372,178.576482,150.884097,1.243385,1.627610,1.433857,1.251932,1.107180,165.960067,147.856207,157.989833,161.973644,164.175074,162.077837,-0.793969,1.616018,-0.948455,0.252029,37.854935,284.355364,478.282996,2270.460414,2757.360945,8.048439,...,33.463807,0.597568,99,146,199,271,347,431,544,691,3670,4.174387,4.644391,5.036953,5.342334,5.659482,5.908083,6.146329,6.378426,6.626718,6.797940,8.279190,7,0.065662,6.700731,7.653969,8.595080,9.554852,10.503395,11.468294,12.421914,13.391032,204.066919,7.893945

In [ ]:
df_test.to_csv('M_sweet_test_boruta.csv', index = False)

Reading Padel Data :

In [ ]:
bitter_train_P = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Boruta/Padel/padel_bitter_train_boruta.csv")
bitter_test_P = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Boruta/Padel/padel_bitter_test_boruta.csv")
sweet_train_P = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Boruta/Padel/sweet_train_boruta.csv")
sweet_test_P = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Boruta/Padel/sweet_test_boruta.csv")

Reading Mordred Data :

In [ ]:
bitter_train_M = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Boruta/Modred/M_bitter_train_boruta.csv")
bitter_test_M = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Boruta/Modred/M_bitter_test_boruta.csv")
sweet_train_M = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Boruta/Modred/M_sweet_train_boruta.csv")
sweet_test_M = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Boruta/Modred/M_sweet_test_boruta.csv")

In [ ]:
sweet_train_M

,ATS2dv,ATS3s,ATS4s,ATS0Z,ATS0m,ATS3m,ATS0p,AATS0d,AATS3d,AATS4d,AATS0Z,AATS2Z,AATS0m,AATS1m,AATS0v,AATS1v,AATS2v,AATS3v,AATS4v,AATS0p,AATS1p,AATS2p,AATS3p,AATS4p,AATS0i,AATS1i,AATS3i,AATS4i,AATS5i,AATS7i,ATSC2c,ATSC3c,ATSC4c,ATSC5c,ATSC1d,ATSC0s,ATSC6Z,ATSC6m,ATSC6v,ATSC3se,...,MID_O,AMID_O,MPC2,MPC3,MPC4,MPC5,MPC6,MPC7,MPC8,MPC9,TMPC10,piPC1,piPC2,piPC3,piPC4,piPC5,piPC6,piPC7,piPC8,piPC9,piPC10,TpiPC10,nRing,JGI3,MWC03,MWC04,MWC05,MWC06,MWC07,MWC08,MWC09,MWC10,TMWC10,SRW06,SRW08,SRW10,AMW,WPol,Zagreb2,Sweet
0,532.000000,652.500000,629.083333,1158.0,4569.170871,9413.865359,50.323528,3.155556,2.830645,2.541353,25.733333,23.192771,101.537130,96.335918,181.029861,233.387037,201.265826,160.826674,144.999343,1.118301,1.445816,1.258039,1.017477,0.952669,169.548550,153.042175,166.058828,167.454386,169.891248,168.161237,-0.652482,1.204605,-0.670753,0.228861,12.975309,160.554938,-5.453827,-19.969966,125.602063,3.369352,...,20.160229,0.876532,36,51,62,73,74,70,70,68,608,3.218876,3.610918,3.951244,4.143135,4.304065,4.317488,4.262680,4.262680,4.234107,4.060443,6.411818,2,0.082526,5.686975,6.602588,7.513709,8.434464,9.350189,10.272738,11.190528,12.113908,122.960890,6.849066,8.525360,10.247042,7.602582,43,147.0,True
1,477.111111,509.601852,424.106996,1830.0,7568.326176,11909.663100,61.317277,3.309524,2.895652,2.603306,43.571429,27.100000,180.198242,118.231375,212.281739,255.059582,215.170710,179.685807,165.227306,1.459935,1.669930,1.435949,1.191106,1.173424,167.215625,150.288731,164.818253,165.667994,168.181859,166.072550,-0.461902,0.767067,-0.461966,0.177657,10.282880,111.512642,-8.217687,-35.280625,-9.450500,2.211955,...,14.925656,0.648942,36,51,62,73,74,70,70,68,608,3.218876,3.610918,3.951244,4.143135,4.304065,4.317488,4.262680,4.262680,4.234107,4.060443,6.411818,2,0.082526,5.686975,6.602588,7.513709,8.434464,9.350189,10.272738,11.190528,12.113908,122.960890,6.849066,8.525360,10.247042,9.428918,43,147.0,True
2,348.000000,327.027778,372.666667,940.0,3710.218949,5232.341659,52.683652,2.923077,2.697674,2.467391,24.102564,20.676923,95.133819,92.739769,206.598109,251.559891,206.377537,173.668198,157.736959,1.350863,1.675350,1.429224,1.235235,1.107106,165.471161,150.112578,161.777305,165.699596,164.954150,163.395314,0.145983,0.338385,-0.698717,0.537398,4.692308,117.287749,66.000000,316.956515,387.214257,-0.360156,...,8.731863,0.415803,27,30,33,37,32,31,29,24,301,3.332205,3.749504,4.020877,4.312476,4.722398,4.452165,4.599529,4.819273,4.884694,4.627421,6.767271,1,0.043981,5.351858,6.175867,6.978214,7.805882,8.618485,9.447229,10.264966,11.093995,112.311208,6.447306,7.951207,9.493487,7.541579,27,105.0,True
3,254.000000,393.500000,344.166667,612.0,2413.585500,4575.372616,25.927979,3.000000,2.460317,1.772727,25.500000,23.047619,100.566063,92.993879,175.518792,227.079812,198.191463,153.270562,114.014016,1.080332,1.407855,1.237989,0.975734,0.796130,170.520236,154.381429,166.741525,172.134683,178.032893,184.993831,-0.307929,0.683571,-0.379936,-0.025630,6.000000,95.268229,37.000000,176.254520,243.836186,1.606344,...,10.677995,0.889833,18,23,22,23,17,7,1,0,135,2.564949,2.944439,3.178054,3.135494,3.178054,2.890372,2.079442,0.693147,0.000000,0.000000,4.912655,1,0.065625,4.962845,5.849325,6.722630,7.610358,8.487764,9.375092,10.253827,11.140687,92.513402,6.163315,7.845024,9.569063,7.502641,20,71.0,True
4,518.000000,682.541667,614.847222,1158.0,4569.170871,9127.853377,50.323528,3.155556,2.661290,2.270677,25.733333,23.192771,101.537130,96.335918,181.029861,233.387037,201.265826,157.905783,132.343321,1.118301,1.445816,1.258039,1.008267,0.897097,169.548550,153.042175,166.015110,169.040621,171.734788,166.939623,-0.628096,1.224730,-0.715289,0.159067,10.975309,160.554938,87.169383,415.124614,512.973197,3.182088,...,20.110824,0.874384,36,49,56,62,58,53,58,67,557,3.218876,3.610918,3.912023,4.043051,4.143135,4.077537,3.988984,4.077537,4.219508,4.276666,6.324359,2,0.077405,5.673323,6.575076,7.467942,8.371242,9.267193,10.170725,11.068184,11.971710,122.361186,6.836259,

# **CHEM_TASTE_DB**

---



In [ ]:
def findPredictionsModred(x_Train, x_Test, y_Train, y_Test):

  #Preparing data for PCA
  pca2 = PCA(n_components=2)
  x_pca2_data_train = pca2.fit_transform(x_Train)
  x_pca2_data_test = pca2.fit_transform(x_Test)

  pca10 = PCA(n_components=10)
  x_pca10_data_train = pca10.fit_transform(x_Train)
  x_pca10_data_test = pca10.fit_transform(x_Test)

  
  
  #Preparing data for Correlation matrix on features
  train_data = pd.concat([x_Train,y_Train],axis=1)
  test_data = pd.concat([x_Test,y_Test],axis=1)
  train_corr , test_corr = correlation_check(train_data, test_data,0.8)

  
  x_Train_corr = train_corr.drop(['Class taste'],axis=1)
  x_Test_corr = test_corr.drop(['Class taste'],axis=1)

  
  
  #Preparing data for Correlation matrix on features and Labels

  a = x_Train_corr.corr(method ='pearson').abs()
  df_temp = a.head(1)
  df_temp.fillna(0,inplace = True)

  colList = []
  for i in df_temp.columns:
    if(df_temp[i].item() < 0.1):
      colList.append(i)

  for i in colList:
    new_x_train_corr = x_Train_corr.drop(i,axis = 1)

  for i in colList:
    new_x_test_corr = x_Test_corr.drop(i,axis = 1)


  

  #Preparing data for selectkBest
  from sklearn.feature_selection import SelectKBest, f_classif
  
  X_new = SelectKBest(f_classif, k=200).fit(x_Train, y_Train)
  cols = X_new.get_support(indices=True)
  features_df_new_train = x_Train.iloc[:,cols]
  features_df_new_test = x_Test.iloc[:,cols]

  



  #Applying Models : 
  #---------------------

  #1. Logistic Regression :
  from sklearn.linear_model import LogisticRegression

  clfLR1 = LogisticRegression(random_state=0)
  
  print("Logistic Regression : ")
  print("------------------------")
  print()

  
  clfLR_Basic = clfLR1.fit(x_Train, y_Train)
  y_pred_Basic = clfLR_Basic.predict(x_Test)
  print("Basic ==>                          Precision: ",precision_score(y_Test, y_pred_Basic,average='weighted')," Recall: ",recall_score(y_Test, y_pred_Basic,average='weighted'),"  F1: ",f1_score(y_Test, y_pred_Basic, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_Basic))
  print()

  clfLR2 = LogisticRegression(random_state=0)

  clfLRpca2 = clfLR2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfLRpca2.predict(x_pca2_data_test)
  print("PCA with n = 2 ==>                 Precision: ",precision_score(y_Test, y_predPca2, average='weighted')," Recall: ",recall_score(y_Test, y_predPca2, average='weighted'),"  F1: ",f1_score(y_Test, y_predPca2, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_predPca2))
  print()

  clfLR3 = LogisticRegression(random_state=0)

  clfLRpca10 = clfLR3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfLRpca10.predict(x_pca10_data_test)
  print("PCA with n = 10 ==>                Precision: ",precision_score(y_Test, y_predPca10, average='weighted')," Recall: ",recall_score(y_Test, y_predPca10, average='weighted'),"  F1: ",f1_score(y_Test, y_predPca10, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_predPca10))
  print()

  clfLR4 = LogisticRegression(random_state=0)

  clfLR_CMF = clfLR4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfLR_CMF.predict(x_Test_corr)
  print("Correlation Matrix on Features ==> Precision: ",precision_score(y_Test, y_pred_CMF, average='weighted')," Recall: ",recall_score(y_Test, y_pred_CMF, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_CMF, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_CMF))
  print()

  clfLR5 = LogisticRegression(random_state=0)

  clfLR_CMFL = clfLR5.fit(new_x_train_corr, y_Train)
  y_pred_CMFL = clfLR_CMFL.predict(new_x_test_corr)
  print("CorMatrix on Features & Labels ==> Precision: ",precision_score(y_Test, y_pred_CMFL, average='weighted')," Recall: ",recall_score(y_Test, y_pred_CMFL, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_CMFL, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_CMFL))
  print()

  clfLR6 = LogisticRegression(random_state=0)

  clfLR_selectK = clfLR6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfLR_selectK.predict(features_df_new_test)
  print("SelectK best with k = 200 ==>      Precision: ",precision_score(y_Test, y_pred_selectK, average='weighted')," Recall: ",recall_score(y_Test, y_pred_selectK, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_selectK, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_selectK))
  print()

  #=====================================================================================================================================================

  #2. Random forgest :
  from sklearn.ensemble import RandomForestClassifier

  clfRF1 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=20, criterion='entropy', random_state=30)
  
  print("Random Forest : ")
  print("----------------")
  print()

  clfRF_Basic = clfRF1.fit(x_Train, y_Train)
  y_pred_Basic = clfRF_Basic.predict(x_Test)
  print("Basic ==>                          Precision: ",precision_score(y_Test, y_pred_Basic,average='weighted')," Recall: ",recall_score(y_Test, y_pred_Basic,average='weighted'),"  F1: ",f1_score(y_Test, y_pred_Basic, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_Basic))
  print()

  clfRF2 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=20, criterion='entropy', random_state=30)


  clfRFpca2 = clfRF2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfRFpca2.predict(x_pca2_data_test)
  print("PCA with n = 2 ==>                 Precision: ",precision_score(y_Test, y_predPca2, average='weighted')," Recall: ",recall_score(y_Test, y_predPca2, average='weighted'),"  F1: ",f1_score(y_Test, y_predPca2, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_predPca2))
  print()

  clfRF3 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=20, criterion='entropy', random_state=30)


  clfRFpca10 = clfRF3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfRFpca10.predict(x_pca10_data_test)
  print("PCA with n = 10 ==>                Precision: ",precision_score(y_Test, y_predPca10, average='weighted')," Recall: ",recall_score(y_Test, y_predPca10, average='weighted'),"  F1: ",f1_score(y_Test, y_predPca10, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_predPca10))
  print()


  clfRF4 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=20, criterion='entropy', random_state=30)

  clfRF_CMF = clfRF4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfRF_CMF.predict(x_Test_corr)
  print("Correlation Matrix on Features ==> Precision: ",precision_score(y_Test, y_pred_CMF, average='weighted')," Recall: ",recall_score(y_Test, y_pred_CMF, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_CMF, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_CMF))
  print()

  clfRF5 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=20, criterion='entropy', random_state=30)

  clfLR_CMFL = clfRF5.fit(new_x_train_corr, y_Train)
  y_pred_CMFL = clfLR_CMFL.predict(new_x_test_corr)
  print("CorMatrix on Features & Labels ==> Precision: ",precision_score(y_Test, y_pred_CMFL, average='weighted')," Recall: ",recall_score(y_Test, y_pred_CMFL, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_CMFL, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_CMFL))
  print()


  clfRF6 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=20, criterion='entropy', random_state=30)

  clfRF_selectK = clfRF6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfRF_selectK.predict(features_df_new_test)
  print("SelectK best with k = 200 ==>      Precision: ",precision_score(y_Test, y_pred_selectK, average='weighted')," Recall: ",recall_score(y_Test, y_pred_selectK, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_selectK, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_selectK))
  print()



  


  #3. Adaboost
  from sklearn.ensemble import AdaBoostClassifier

  clfAB1 = AdaBoostClassifier(n_estimators=100, random_state=0)

  print("AdaBoost : ")
  print("----------------")
  print()

  clfAB_Basic = clfAB1.fit(x_Train, y_Train)
  y_pred_Basic = clfAB_Basic.predict(x_Test)
  print("Basic ==>                          Precision: ",precision_score(y_Test, y_pred_Basic,average='weighted')," Recall: ",recall_score(y_Test, y_pred_Basic,average='weighted'),"  F1: ",f1_score(y_Test, y_pred_Basic, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_Basic))
  print()

  clfAB2 = AdaBoostClassifier(n_estimators=100, random_state=0)

  clfABpca2 = clfAB2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfABpca2.predict(x_pca2_data_test)
  print("PCA with n = 2 ==>                 Precision: ",precision_score(y_Test, y_predPca2, average='weighted')," Recall: ",recall_score(y_Test, y_predPca2, average='weighted'),"  F1: ",f1_score(y_Test, y_predPca2, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_predPca2))
  print()

  clfAB3 = AdaBoostClassifier(n_estimators=100, random_state=0)

  clfABpca10 = clfAB3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfABpca10.predict(x_pca10_data_test)
  print("PCA with n = 10 ==>                Precision: ",precision_score(y_Test, y_predPca10, average='weighted')," Recall: ",recall_score(y_Test, y_predPca10, average='weighted'),"  F1: ",f1_score(y_Test, y_predPca10, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_predPca10))
  print()


  clfAB4 = AdaBoostClassifier(n_estimators=100, random_state=0)

  clfAB_CMF = clfAB4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfAB_CMF.predict(x_Test_corr)
  print("Correlation Matrix on Features ==> Precision: ",precision_score(y_Test, y_pred_CMF, average='weighted')," Recall: ",recall_score(y_Test, y_pred_CMF, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_CMF, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_CMF))
  print()

  clfAB5 = AdaBoostClassifier(n_estimators=100, random_state=0)

  clfAB_CMFL = clfAB5.fit(new_x_train_corr, y_Train)
  y_pred_CMFL = clfAB_CMFL.predict(new_x_test_corr)
  print("CorMatrix on Features & Labels ==> Precision: ",precision_score(y_Test, y_pred_CMFL, average='weighted')," Recall: ",recall_score(y_Test, y_pred_CMFL, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_CMFL, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_CMFL))
  print()

  clfAB6 = AdaBoostClassifier(n_estimators=100, random_state=0)


  clfAB_selectK = clfAB6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfAB_selectK.predict(features_df_new_test)
  print("SelectK best with k = 200 ==>      Precision: ",precision_score(y_Test, y_pred_selectK, average='weighted')," Recall: ",recall_score(y_Test, y_pred_selectK, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_selectK, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_selectK))
  print()



  
  #4. XGBoostClassifier
  import xgboost as xg

  xgb_r1 = xg.XGBClassifier(max_depth=10)

  print("XGBoostClassifier : ")
  print("----------------")
  print()

  clfXGB_Basic = xgb_r1.fit(x_Train, y_Train)
  y_pred_Basic = clfXGB_Basic.predict(x_Test)
  print("Basic ==>                          Precision: ",precision_score(y_Test, y_pred_Basic,average='weighted')," Recall: ",recall_score(y_Test, y_pred_Basic,average='weighted'),"  F1: ",f1_score(y_Test, y_pred_Basic, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_Basic))
  print()

  xgb_r2 = xg.XGBClassifier(max_depth=20)

  clfXGBpca2 = xgb_r2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfXGBpca2.predict(x_pca2_data_test)
  print("PCA with n = 2 ==>                 Precision: ",precision_score(y_Test, y_predPca2, average='weighted')," Recall: ",recall_score(y_Test, y_predPca2, average='weighted'),"  F1: ",f1_score(y_Test, y_predPca2, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_predPca2))
  print()

  xgb_r3 = xg.XGBClassifier(max_depth=20)


  clfXGBpca10 = xgb_r3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfXGBpca10.predict(x_pca10_data_test)
  print("PCA with n = 10 ==>                Precision: ",precision_score(y_Test, y_predPca10, average='weighted')," Recall: ",recall_score(y_Test, y_predPca10, average='weighted'),"  F1: ",f1_score(y_Test, y_predPca10, average='macro')," Accuracy: ",accuracy_score(y_Test, y_predPca10))
  print()


  xgb_r4 = xg.XGBClassifier(max_depth=20)

  clfXGB_CMF = xgb_r4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfXGB_CMF.predict(x_Test_corr)
  print("Correlation Matrix on Features ==> Precision: ",precision_score(y_Test, y_pred_CMF, average='weighted')," Recall: ",recall_score(y_Test, y_pred_CMF, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_CMF, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_CMF))
  print()

  xgb_r5 = xg.XGBClassifier(max_depth=20)

  clfXGB_CMFL = xgb_r5.fit(new_x_train_corr, y_Train)
  y_pred_CMFL = clfXGB_CMFL.predict(new_x_test_corr)
  print("CorMatrix on Features & Labels ==> Precision: ",precision_score(y_Test, y_pred_CMFL, average='weighted')," Recall: ",recall_score(y_Test, y_pred_CMFL, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_CMFL, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_CMFL))
  print()

  xgb_r6 = xg.XGBClassifier(max_depth=20)

  clfXGB_selectK = xgb_r6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfXGB_selectK.predict(features_df_new_test)
  print("SelectK best with k = 200 ==>      Precision: ",precision_score(y_Test, y_pred_selectK, average='weighted')," Recall: ",recall_score(y_Test, y_pred_selectK, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_selectK, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_selectK))
  print()


  
  

In [ ]:
def findPredictionsPadel(x_Train, x_Test, y_Train, y_Test):

  #Preparing data for PCA
  pca2 = PCA(n_components=2)
  x_pca2_data_train = pca2.fit_transform(x_Train)
  x_pca2_data_test = pca2.fit_transform(x_Test)

  pca10 = PCA(n_components=10)
  x_pca10_data_train = pca10.fit_transform(x_Train)
  x_pca10_data_test = pca10.fit_transform(x_Test)

  
  
  #Preparing data for Correlation matrix on features
  train_data = pd.concat([x_Train,y_Train],axis=1)
  test_data = pd.concat([x_Test,y_Test],axis=1)
  train_corr , test_corr = correlation_check(train_data, test_data,0.8)

  
  x_Train_corr = train_corr.drop(['Class Taste'],axis=1)
  x_Test_corr = test_corr.drop(['Class Taste'],axis=1)

  
  
  #Preparing data for Correlation matrix on features and Labels

  a = x_Train_corr.corr(method ='pearson').abs()
  df_temp = a.head(1)
  df_temp.fillna(0,inplace = True)

  colList = []
  for i in df_temp.columns:
    if(df_temp[i].item() < 0.1):
      colList.append(i)

  for i in colList:
    new_x_train_corr = x_Train_corr.drop(i,axis = 1)

  for i in colList:
    new_x_test_corr = x_Test_corr.drop(i,axis = 1)


  

  #Preparing data for selectkBest
  from sklearn.feature_selection import SelectKBest, f_classif
  
  X_new = SelectKBest(f_classif, k=200).fit(x_Train, y_Train)
  cols = X_new.get_support(indices=True)
  features_df_new_train = x_Train.iloc[:,cols]
  features_df_new_test = x_Test.iloc[:,cols]

  



  #Applying Models : 
  #---------------------

  #1. Logistic Regression :
  from sklearn.linear_model import LogisticRegression

  clfLR1 = LogisticRegression(random_state=0)
  
  print("Logistic Regression : ")
  print("------------------------")
  print()

  
  clfLR_Basic = clfLR1.fit(x_Train, y_Train)
  y_pred_Basic = clfLR_Basic.predict(x_Test)
  print("Basic ==>                          Precision: ",precision_score(y_Test, y_pred_Basic,average='weighted')," Recall: ",recall_score(y_Test, y_pred_Basic,average='weighted'),"  F1: ",f1_score(y_Test, y_pred_Basic, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_Basic))
  print()

  clfLR2 = LogisticRegression(random_state=0)

  clfLRpca2 = clfLR2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfLRpca2.predict(x_pca2_data_test)
  print("PCA with n = 2 ==>                 Precision: ",precision_score(y_Test, y_predPca2, average='weighted')," Recall: ",recall_score(y_Test, y_predPca2, average='weighted'),"  F1: ",f1_score(y_Test, y_predPca2, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_predPca2))
  print()

  clfLR3 = LogisticRegression(random_state=0)

  clfLRpca10 = clfLR3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfLRpca10.predict(x_pca10_data_test)
  print("PCA with n = 10 ==>                Precision: ",precision_score(y_Test, y_predPca10, average='weighted')," Recall: ",recall_score(y_Test, y_predPca10, average='weighted'),"  F1: ",f1_score(y_Test, y_predPca10, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_predPca10))
  print()

  clfLR4 = LogisticRegression(random_state=0)

  clfLR_CMF = clfLR4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfLR_CMF.predict(x_Test_corr)
  print("Correlation Matrix on Features ==> Precision: ",precision_score(y_Test, y_pred_CMF, average='weighted')," Recall: ",recall_score(y_Test, y_pred_CMF, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_CMF, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_CMF))
  print()

  clfLR5 = LogisticRegression(random_state=0)

  clfLR_CMFL = clfLR5.fit(new_x_train_corr, y_Train)
  y_pred_CMFL = clfLR_CMFL.predict(new_x_test_corr)
  print("CorMatrix on Features & Labels ==> Precision: ",precision_score(y_Test, y_pred_CMFL, average='weighted')," Recall: ",recall_score(y_Test, y_pred_CMFL, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_CMFL, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_CMFL))
  print()

  clfLR6 = LogisticRegression(random_state=0)

  clfLR_selectK = clfLR6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfLR_selectK.predict(features_df_new_test)
  print("SelectK best with k = 200 ==>      Precision: ",precision_score(y_Test, y_pred_selectK, average='weighted')," Recall: ",recall_score(y_Test, y_pred_selectK, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_selectK, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_selectK))
  print()

  #=====================================================================================================================================================

  #2. Random forgest :
  from sklearn.ensemble import RandomForestClassifier

  clfRF1 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=25, criterion='entropy', random_state=30)
  
  print("Random Forest : ")
  print("----------------")
  print()

  clfRF_Basic = clfRF1.fit(x_Train, y_Train)
  y_pred_Basic = clfRF_Basic.predict(x_Test)
  print("Basic ==>                          Precision: ",precision_score(y_Test, y_pred_Basic,average='weighted')," Recall: ",recall_score(y_Test, y_pred_Basic,average='weighted'),"  F1: ",f1_score(y_Test, y_pred_Basic, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_Basic))
  print()

  clfRF2 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=25, criterion='entropy', random_state=30)


  clfRFpca2 = clfRF2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfRFpca2.predict(x_pca2_data_test)
  print("PCA with n = 2 ==>                 Precision: ",precision_score(y_Test, y_predPca2, average='weighted')," Recall: ",recall_score(y_Test, y_predPca2, average='weighted'),"  F1: ",f1_score(y_Test, y_predPca2, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_predPca2))
  print()

  clfRF3 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=25, criterion='entropy', random_state=30)


  clfRFpca10 = clfRF3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfRFpca10.predict(x_pca10_data_test)
  print("PCA with n = 10 ==>                Precision: ",precision_score(y_Test, y_predPca10, average='weighted')," Recall: ",recall_score(y_Test, y_predPca10, average='weighted'),"  F1: ",f1_score(y_Test, y_predPca10, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_predPca10))
  print()


  clfRF4 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=25, criterion='entropy', random_state=30)

  clfRF_CMF = clfRF4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfRF_CMF.predict(x_Test_corr)
  print("Correlation Matrix on Features ==> Precision: ",precision_score(y_Test, y_pred_CMF, average='weighted')," Recall: ",recall_score(y_Test, y_pred_CMF, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_CMF, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_CMF))
  print()

  clfRF5 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=25, criterion='entropy', random_state=30)

  clfLR_CMFL = clfRF5.fit(new_x_train_corr, y_Train)
  y_pred_CMFL = clfLR_CMFL.predict(new_x_test_corr)
  print("CorMatrix on Features & Labels ==> Precision: ",precision_score(y_Test, y_pred_CMFL, average='weighted')," Recall: ",recall_score(y_Test, y_pred_CMFL, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_CMFL, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_CMFL))
  print()


  clfRF6 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=25, criterion='entropy', random_state=30)

  clfRF_selectK = clfRF6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfRF_selectK.predict(features_df_new_test)
  print("SelectK best with k = 200 ==>      Precision: ",precision_score(y_Test, y_pred_selectK, average='weighted')," Recall: ",recall_score(y_Test, y_pred_selectK, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_selectK, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_selectK))
  print()



  


  #3. Adaboost
  from sklearn.ensemble import AdaBoostClassifier

  clfAB1 = AdaBoostClassifier(n_estimators=100, random_state=0)

  print("AdaBoost : ")
  print("----------------")
  print()

  clfAB_Basic = clfAB1.fit(x_Train, y_Train)
  y_pred_Basic = clfAB_Basic.predict(x_Test)
  print("Basic ==>                          Precision: ",precision_score(y_Test, y_pred_Basic,average='weighted')," Recall: ",recall_score(y_Test, y_pred_Basic,average='weighted'),"  F1: ",f1_score(y_Test, y_pred_Basic, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_Basic))
  print()

  clfAB2 = AdaBoostClassifier(n_estimators=100, random_state=0)

  clfABpca2 = clfAB2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfABpca2.predict(x_pca2_data_test)
  print("PCA with n = 2 ==>                 Precision: ",precision_score(y_Test, y_predPca2, average='weighted')," Recall: ",recall_score(y_Test, y_predPca2, average='weighted'),"  F1: ",f1_score(y_Test, y_predPca2, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_predPca2))
  print()

  clfAB3 = AdaBoostClassifier(n_estimators=100, random_state=0)

  clfABpca10 = clfAB3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfABpca10.predict(x_pca10_data_test)
  print("PCA with n = 10 ==>                Precision: ",precision_score(y_Test, y_predPca10, average='weighted')," Recall: ",recall_score(y_Test, y_predPca10, average='weighted'),"  F1: ",f1_score(y_Test, y_predPca10, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_predPca10))
  print()


  clfAB4 = AdaBoostClassifier(n_estimators=100, random_state=0)

  clfAB_CMF = clfAB4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfAB_CMF.predict(x_Test_corr)
  print("Correlation Matrix on Features ==> Precision: ",precision_score(y_Test, y_pred_CMF, average='weighted')," Recall: ",recall_score(y_Test, y_pred_CMF, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_CMF, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_CMF))
  print()

  clfAB5 = AdaBoostClassifier(n_estimators=100, random_state=0)

  clfAB_CMFL = clfAB5.fit(new_x_train_corr, y_Train)
  y_pred_CMFL = clfAB_CMFL.predict(new_x_test_corr)
  print("CorMatrix on Features & Labels ==> Precision: ",precision_score(y_Test, y_pred_CMFL, average='weighted')," Recall: ",recall_score(y_Test, y_pred_CMFL, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_CMFL, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_CMFL))
  print()

  clfAB6 = AdaBoostClassifier(n_estimators=100, random_state=0)


  clfAB_selectK = clfAB6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfAB_selectK.predict(features_df_new_test)
  print("SelectK best with k = 200 ==>      Precision: ",precision_score(y_Test, y_pred_selectK, average='weighted')," Recall: ",recall_score(y_Test, y_pred_selectK, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_selectK, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_selectK))
  print()



  
  #4. XGBoostClassifier
  import xgboost as xg

  xgb_r1 = xg.XGBClassifier(max_depth=20)

  print("XGBoostClassifier : ")
  print("----------------")
  print()

  clfXGB_Basic = xgb_r1.fit(x_Train, y_Train)
  y_pred_Basic = clfXGB_Basic.predict(x_Test)
  print("Basic ==>                          Precision: ",precision_score(y_Test, y_pred_Basic,average='weighted')," Recall: ",recall_score(y_Test, y_pred_Basic,average='weighted'),"  F1: ",f1_score(y_Test, y_pred_Basic, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_Basic))
  print()

  xgb_r2 = xg.XGBClassifier(max_depth=20)

  clfXGBpca2 = xgb_r2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfXGBpca2.predict(x_pca2_data_test)
  print("PCA with n = 2 ==>                 Precision: ",precision_score(y_Test, y_predPca2, average='weighted')," Recall: ",recall_score(y_Test, y_predPca2, average='weighted'),"  F1: ",f1_score(y_Test, y_predPca2, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_predPca2))
  print()

  xgb_r3 = xg.XGBClassifier(max_depth=20)


  clfXGBpca10 = xgb_r3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfXGBpca10.predict(x_pca10_data_test)
  print("PCA with n = 10 ==>                Precision: ",precision_score(y_Test, y_predPca10, average='weighted')," Recall: ",recall_score(y_Test, y_predPca10, average='weighted'),"  F1: ",f1_score(y_Test, y_predPca10, average='macro')," Accuracy: ",accuracy_score(y_Test, y_predPca10))
  print()


  xgb_r4 = xg.XGBClassifier(max_depth=20)

  clfXGB_CMF = xgb_r4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfXGB_CMF.predict(x_Test_corr)
  print("Correlation Matrix on Features ==> Precision: ",precision_score(y_Test, y_pred_CMF, average='weighted')," Recall: ",recall_score(y_Test, y_pred_CMF, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_CMF, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_CMF))
  print()

  xgb_r5 = xg.XGBClassifier(max_depth=20)

  clfXGB_CMFL = xgb_r5.fit(new_x_train_corr, y_Train)
  y_pred_CMFL = clfXGB_CMFL.predict(new_x_test_corr)
  print("CorMatrix on Features & Labels ==> Precision: ",precision_score(y_Test, y_pred_CMFL, average='weighted')," Recall: ",recall_score(y_Test, y_pred_CMFL, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_CMFL, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_CMFL))
  print()

  xgb_r6 = xg.XGBClassifier(max_depth=20)

  clfXGB_selectK = xgb_r6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfXGB_selectK.predict(features_df_new_test)
  print("SelectK best with k = 200 ==>      Precision: ",precision_score(y_Test, y_pred_selectK, average='weighted')," Recall: ",recall_score(y_Test, y_pred_selectK, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_selectK, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_selectK))
  print()


  
  

In [ ]:
def correlation_check(traindata,testdata,thresh): # drop columns above certain threshold    
        corr_matrix = traindata.corr()
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
        to_drop = [column for column in upper.columns if any(upper[column] >thresh)]
        trainset=traindata.drop(traindata[to_drop], axis=1)
        testset=testdata.drop(testdata[to_drop],axis=1)
        return trainset,testset

In [ ]:
def gridSearchOnLogReg(x, y):
  grid={
        "C":np.logspace(-3,3,7,1,0),
        "penalty":["l1","l2","elasticnet","none"],
        "max_iter":[50,80,100],
        "solver":["newton-cg", "lbfgs", "liblinear", "sag", "saga"]
        }
  logreg=LogisticRegression()
  logreg_cv=GridSearchCV(logreg,grid)
  logreg_cv.fit(x,y)
  return logreg_cv

**MORDRED DATA**

In [ ]:
chemTasteDB = pd.read_csv("/content/drive/MyDrive/Capstone/chemTasteDB/Mordred Cleaned/chemTasteDBCleaned.csv")

In [ ]:
y = chemTasteDB['Class taste']
x = chemTasteDB.drop(['Class taste','Name','Taste'],axis=1)
y=y.to_frame()
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [ ]:
findPredictionsModred(x_train, x_test, y_train, y_test)

Logistic Regression : 
------------------------

Basic ==>                          Precision:  0.29935297681348005  Recall:  0.37907608695652173   F1:  0.3277825252691276  Accuracy:  0.37907608695652173

PCA with n = 2 ==>                 Precision:  0.22755819947655007  Recall:  0.35054347826086957   F1:  0.2448037247309764  Accuracy:  0.35054347826086957

PCA with n = 10 ==>                Precision:  0.2283682870397116  Recall:  0.3491847826086957   F1:  0.24636972180176422  Accuracy:  0.3491847826086957

Correlation Matrix on Features ==> Precision:  0.24888978403450093  Recall:  0.35597826086956524   F1:  0.25339010682190904  Accuracy:  0.35597826086956524

CorMatrix on Features & Labels ==> Precision:  0.24888978403450093  Recall:  0.35597826086956524   F1:  0.25339010682190904  Accuracy:  0.35597826086956524

SelectK best with k = 200 ==>      Precision:  0.14784860881379963  Recall:  0.3845108695652174   F1:  0.21357522293808937  Accuracy:  0.3845108695652174

Random Forest : 

**PADEL DATA**

In [ ]:
chemTasteDB = pd.read_csv("/content/drive/MyDrive/Capstone/chemTasteDB/Padel/ChemTasteDBFeatures.csv", encoding='unicode_escape')

In [ ]:
data =chemTasteDB[~chemTasteDB.isin([np.nan, np.inf, -np.inf]).any(1)]

In [ ]:
y = data['Class Taste']
x = data.drop(['Class Taste','Name','Taste', 'ID'],axis=1)
y=y.to_frame()
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [ ]:
findPredictionsPadel(x_train, x_test, y_train, y_test)

Logistic Regression : 
------------------------

Basic ==>                          Precision:  0.39645203218505093  Recall:  0.44613259668508287   F1:  0.3361711367998856  Accuracy:  0.44613259668508287

PCA with n = 2 ==>                 Precision:  0.4935509694154198  Recall:  0.356353591160221   F1:  0.2118467725171207  Accuracy:  0.356353591160221

PCA with n = 10 ==>                Precision:  0.5244689464659935  Recall:  0.35359116022099446   F1:  0.20682172274119687  Accuracy:  0.35359116022099446

Correlation Matrix on Features ==> Precision:  0.3552551486075432  Recall:  0.4488950276243094   F1:  0.36711142410101555  Accuracy:  0.4488950276243094

CorMatrix on Features & Labels ==> Precision:  0.35809962510540566  Recall:  0.4530386740331492   F1:  0.37078868259970044  Accuracy:  0.4530386740331492

SelectK best with k = 200 ==>      Precision:  0.5424095455082724  Recall:  0.6077348066298343   F1:  0.5649914516379849  Accuracy:  0.6077348066298343

Random Forest : 
---------

# **Extended chemTasteDB**

### **MODRED**

**Without K-fold**

In [ ]:
chemTasteDB = pd.read_csv("/content/drive/MyDrive/Capstone/ChemTasteDB + Sweet_Bitter/chemTasteDBModredMerged.csv")

In [ ]:
y = chemTasteDB['Class taste']
x = chemTasteDB.drop(['Class taste'],axis=1)
y=y.to_frame()
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [ ]:
findPredictionsModred(x_train, x_test, y_train, y_test)

Logistic Regression : 
------------------------

Basic ==>                          Precision:  0.3629957962000359  Recall:  0.41112828438948995   F1:  0.30287245823650744  Accuracy:  0.41112828438948995

PCA with n = 2 ==>                 Precision:  0.42208043381705734  Recall:  0.38330757341576505   F1:  0.260667500329973  Accuracy:  0.38330757341576505

PCA with n = 10 ==>                Precision:  0.36282044203696795  Recall:  0.3894899536321484   F1:  0.2732927308956863  Accuracy:  0.3894899536321484

Correlation Matrix on Features ==> Precision:  0.35928689489160176  Recall:  0.4428129829984544   F1:  0.3441086036665524  Accuracy:  0.4428129829984544

CorMatrix on Features & Labels ==> Precision:  0.37029925965203453  Recall:  0.4435857805255023   F1:  0.34555284822529764  Accuracy:  0.4435857805255023

SelectK best with k = 200 ==>      Precision:  0.16223428734680023  Recall:  0.4026275115919629   F1:  0.2312777657545895  Accuracy:  0.4026275115919629

Random Forest : 
------

### **PADEL**

**Without K-fold**

In [ ]:
chemTasteDB = pd.read_csv("/content/drive/MyDrive/Capstone/ChemTasteDB + Sweet_Bitter/chemTasteDBPadelMerged.csv")

In [ ]:
chemTasteDB = chemTasteDB.dropna()

In [ ]:
y = chemTasteDB['Class Taste']
x = chemTasteDB.drop(['Class Taste'],axis=1)
y=y.to_frame()
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [ ]:
findPredictionsPadel(x_train, x_test, y_train, y_test)

Logistic Regression : 
------------------------

Basic ==>                          Precision:  0.5360074196262363  Recall:  0.592280701754386   F1:  0.5376832577431835  Accuracy:  0.592280701754386

PCA with n = 2 ==>                 Precision:  0.4080666983173854  Recall:  0.44280701754385965   F1:  0.2919552964746166  Accuracy:  0.44280701754385965

PCA with n = 10 ==>                Precision:  0.3080519034069522  Recall:  0.42105263157894735   F1:  0.28302906915842246  Accuracy:  0.42105263157894735

Correlation Matrix on Features ==> Precision:  0.48369354904616535  Recall:  0.5347368421052632   F1:  0.48409065234754733  Accuracy:  0.5347368421052632

CorMatrix on Features & Labels ==> Precision:  0.45660340023848794  Recall:  0.5228070175438596   F1:  0.47845932032512906  Accuracy:  0.5228070175438596

SelectK best with k = 200 ==>      Precision:  0.5905194047299355  Recall:  0.6133333333333333   F1:  0.5851614459771779  Accuracy:  0.6133333333333333

Random Forest : 
---------

### **With K-fold**

**Mordred**

In [ ]:
chemTasteDBM = pd.read_csv("/content/drive/MyDrive/Capstone/ChemTasteDB + Sweet_Bitter/chemTasteDBModredMerged.csv")

In [ ]:
chemTasteDBM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5175 entries, 0 to 5174
Columns: 1614 entries, ABC to Class taste
dtypes: bool(2), float64(1301), int64(310), object(1)
memory usage: 63.7+ MB


In [ ]:
Y = chemTasteDBM['Class taste']
X = chemTasteDBM.drop(['Class taste'],axis=1)

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5175 entries, 0 to 5174
Columns: 1613 entries, ABC to mZagreb2
dtypes: bool(2), float64(1301), int64(310)
memory usage: 63.6 MB


In [ ]:
Y=Y.to_frame()

In [ ]:
Y

,Class taste
0,Sweetness
1,Sweetness
2,Sweetness
3,Sweetness
4,Miscellaneous
...,...
5170,Bitterness
5171,Bitterness
5172,Sweetness
5173,Bitterness


In [ ]:
x = X.to_numpy()
y = Y.to_numpy()

In [ ]:
import numpy as np
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)

list_LR_Pre_Basic = []
list_LR_f1_Basic = []
list_LR_Acc_Basic = []

list_LR_Pre_PCA2 = []
list_LR_f1_PCA2 = []
list_LR_Acc_PCA2 = []

list_LR_Pre_PCA10 = []
list_LR_f1_PCA10 = []
list_LR_Acc_PCA10 = []

list_LR_Pre_CM = []
list_LR_f1_CM = []
list_LR_Acc_CM = []

list_LR_Pre_CMFL = []
list_LR_f1_CMFL = []
list_LR_Acc_CMFL = []

list_LR_Pre_SK = []
list_LR_f1_SK = []
list_LR_Acc_SK = []

#==============================

list_RF_Pre_Basic = []
list_RF_f1_Basic = []
list_RF_Acc_Basic = []

list_RF_Pre_PCA2 = []
list_RF_f1_PCA2 = []
list_RF_Acc_PCA2 = []

list_RF_Pre_PCA10 = []
list_RF_f1_PCA10 = []
list_RF_Acc_PCA10 = []

list_RF_Pre_CM = []
list_RF_f1_CM = []
list_RF_Acc_CM = []

list_RF_Pre_CMFL = []
list_RF_f1_CMFL = []
list_RF_Acc_CMFL = []

list_RF_Pre_SK = []
list_RF_f1_SK = []
list_RF_Acc_SK = []


#==============================

list_AB_Pre_Basic = []
list_AB_f1_Basic = []
list_AB_Acc_Basic = []

list_AB_Pre_PCA2 = []
list_AB_f1_PCA2 = []
list_AB_Acc_PCA2 = []

list_AB_Pre_PCA10 = []
list_AB_f1_PCA10 = []
list_AB_Acc_PCA10 = []

list_AB_Pre_CM = []
list_AB_f1_CM = []
list_AB_Acc_CM = []

list_AB_Pre_CMFL = []
list_AB_f1_CMFL = []
list_AB_Acc_CMFL = []

list_AB_Pre_SK = []
list_AB_f1_SK = []
list_AB_Acc_SK = []


#==============================

list_XG_Pre_Basic = []
list_XG_f1_Basic = []
list_XG_Acc_Basic = []

list_XG_Pre_PCA2 = []
list_XG_f1_PCA2 = []
list_XG_Acc_PCA2 = []

list_XG_Pre_PCA10 = []
list_XG_f1_PCA10 = []
list_XG_Acc_PCA10 = []

list_XG_Pre_CM = []
list_XG_f1_CM = []
list_XG_Acc_CM = []

list_XG_Pre_CMFL = []
list_XG_f1_CMFL = []
list_XG_Acc_CMFL = []

list_XG_Pre_SK = []
list_XG_f1_SK = []
list_XG_Acc_SK = []


for train_index, test_index in kf.split(x):
  x_Train, x_Test = x[train_index], x[test_index]
  y_Train, y_Test = y[train_index], y[test_index]


  #Preparing data for PCA
  pca2 = PCA(n_components=2)
  x_pca2_data_train = pca2.fit_transform(x_Train)
  x_pca2_data_test = pca2.fit_transform(x_Test)

  pca10 = PCA(n_components=10)
  x_pca10_data_train = pca10.fit_transform(x_Train)
  x_pca10_data_test = pca10.fit_transform(x_Test)

  
  #Preparing data for Correlation matrix on features
  x_Train = pd.DataFrame(x_Train,columns = X.columns)
  y_Train = pd.DataFrame(y_Train,columns = Y.columns)
  x_Test = pd.DataFrame(x_Test,columns = X.columns)
  y_Test = pd.DataFrame(y_Test,columns = Y.columns)

  train_data = pd.concat([x_Train,y_Train],axis=1)
  test_data = pd.concat([x_Test,y_Test],axis=1)
  train_corr , test_corr = correlation_check(train_data, test_data,0.8)


  x_Train_corr = train_corr.drop(['Class taste'],axis=1)
  x_Test_corr = test_corr.drop(['Class taste'],axis=1)

  
  
  #Preparing data for Correlation matrix on features and Labels

  a = x_Train_corr.corr(method ='pearson').abs()
  df_temp = a.head(1)
  df_temp.fillna(0,inplace = True)

  colList = []
  for i in df_temp.columns:
    if(df_temp[i].item() < 0.2):
      colList.append(i)

  for i in colList:
    new_x_train_corr = x_Train_corr.drop(i,axis = 1)
    new_x_test_corr = x_Test_corr.drop(i,axis = 1)
  

  #Preparing data for selectkBest
  from sklearn.feature_selection import SelectKBest, f_classif
  
  X_new = SelectKBest(f_classif, k=200).fit(x_Train, y_Train)
  cols = X_new.get_support(indices=True)
  features_df_new_train = x_Train.iloc[:,cols]
  features_df_new_test = x_Test.iloc[:,cols]


  x_Train = x_Train.to_numpy()
  x_Test = x_Test.to_numpy()
  y_Train = y_Train.to_numpy()
  y_Test = y_Test.to_numpy()

  # Logistic Regression
  from sklearn.linear_model import LogisticRegression

  clfLR1 = LogisticRegression(random_state=0)

  clfLR_Basic = clfLR1.fit(x_Train, y_Train)
  y_pred_Basic = clfLR_Basic.predict(x_Test)

  list_LR_Pre_Basic.append(precision_score(y_Test, y_pred_Basic,average='weighted'))
  list_LR_f1_Basic.append(f1_score(y_Test, y_pred_Basic, average='weighted'))
  list_LR_Acc_Basic.append(accuracy_score(y_Test, y_pred_Basic))
  
  clfLR2 = LogisticRegression(random_state=0)

  clfLRpca2 = clfLR2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfLRpca2.predict(x_pca2_data_test)

  list_LR_Pre_PCA2.append(precision_score(y_Test, y_predPca2, average='weighted'))
  list_LR_f1_PCA2.append(f1_score(y_Test, y_predPca2, average='weighted'))
  list_LR_Acc_PCA2.append(accuracy_score(y_Test, y_predPca2))


  clfLR3 = LogisticRegression(random_state=0)

  clfLRpca10 = clfLR3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfLRpca10.predict(x_pca10_data_test)
 
  list_LR_Pre_PCA10.append(precision_score(y_Test, y_predPca10, average='weighted'))
  list_LR_f1_PCA10.append(f1_score(y_Test, y_predPca10, average='weighted'))
  list_LR_Acc_PCA10.append(accuracy_score(y_Test, y_predPca10))


  clfLR4 = LogisticRegression(random_state=0)

  clfLR_CMF = clfLR4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfLR_CMF.predict(x_Test_corr)
  
  list_LR_Pre_CM.append(precision_score(y_Test, y_pred_CMF, average='weighted'))
  list_LR_f1_CM.append(f1_score(y_Test, y_pred_CMF, average='weighted'))
  list_LR_Acc_CM.append(accuracy_score(y_Test, y_pred_CMF))

  clfLR5 = LogisticRegression(random_state=0)

  # clfLR_CMFL = clfLR5.fit(new_x_train_corr, y_Train)
  # y_pred_CMFL = clfLR_CMFL.predict(new_x_test_corr)
  
  # list_LR_Pre_CMFL.append(precision_score(y_Test, y_pred_CMFL, average='weighted'))
  # list_LR_f1_CMFL.append(f1_score(y_Test, y_pred_CMFL, average='weighted'))
  # list_LR_Acc_CMFL.append(accuracy_score(y_Test, y_pred_CMFL))

  clfLR6 = LogisticRegression(random_state=0)

  clfLR_selectK = clfLR6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfLR_selectK.predict(features_df_new_test)

  list_LR_Pre_SK.append(precision_score(y_Test, y_pred_selectK, average='weighted'))
  list_LR_f1_SK.append(f1_score(y_Test, y_pred_selectK, average='weighted'))
  list_LR_Acc_SK.append(accuracy_score(y_Test, y_pred_selectK))

  #=====================================================================================================================================================

  #2. Random forgest :
  from sklearn.ensemble import RandomForestClassifier

  clfRF1 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=25, criterion='entropy', random_state=30)
  
  

  clfRF_Basic = clfRF1.fit(x_Train, y_Train)
  y_pred_Basic = clfRF_Basic.predict(x_Test)
  list_RF_Pre_Basic.append(precision_score(y_Test, y_pred_Basic,average='weighted'))
  list_RF_f1_Basic.append(f1_score(y_Test, y_pred_Basic, average='weighted'))
  list_RF_Acc_Basic.append(accuracy_score(y_Test, y_pred_Basic))

  clfRF2 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=25, criterion='entropy', random_state=30)


  clfRFpca2 = clfRF2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfRFpca2.predict(x_pca2_data_test)
  list_RF_Pre_PCA2.append(precision_score(y_Test, y_predPca2, average='weighted'))
  list_RF_f1_PCA2.append(f1_score(y_Test, y_predPca2, average='weighted'))
  list_RF_Acc_PCA2.append(accuracy_score(y_Test, y_predPca2))

  clfRF3 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=25, criterion='entropy', random_state=30)


  clfRFpca10 = clfRF3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfRFpca10.predict(x_pca10_data_test)
  list_RF_Pre_PCA10.append(precision_score(y_Test, y_predPca10, average='weighted'))
  list_RF_f1_PCA10.append(f1_score(y_Test, y_predPca10, average='weighted'))
  list_RF_Acc_PCA10.append(accuracy_score(y_Test, y_predPca10))


  clfRF4 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=25, criterion='entropy', random_state=30)

  clfRF_CMF = clfRF4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfRF_CMF.predict(x_Test_corr)
  list_RF_Pre_CM.append(precision_score(y_Test, y_pred_CMF, average='weighted'))
  list_RF_f1_CM.append(f1_score(y_Test, y_pred_CMF, average='weighted'))
  list_RF_Acc_CM.append(accuracy_score(y_Test, y_pred_CMF))

  clfRF5 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=25, criterion='entropy', random_state=30)

  # clfLR_CMFL = clfRF5.fit(new_x_train_corr, y_Train)
  # y_pred_CMFL = clfLR_CMFL.predict(new_x_test_corr)
  # list_RF_Pre_CMFL.append(precision_score(y_Test, y_pred_CMFL, average='weighted'))
  # list_RF_f1_CMFL.append(f1_score(y_Test, y_pred_CMFL, average='weighted'))
  # list_RF_Acc_CMFL.append(accuracy_score(y_Test, y_pred_CMFL))

  clfRF6 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=25, criterion='entropy', random_state=30)

  clfRF_selectK = clfRF6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfRF_selectK.predict(features_df_new_test)
  list_RF_Pre_SK.append(precision_score(y_Test, y_pred_selectK, average='weighted'))
  list_RF_f1_SK.append(f1_score(y_Test, y_pred_selectK, average='weighted'))
  list_RF_Acc_SK.append(accuracy_score(y_Test, y_pred_selectK))



  


  #3. Adaboost
  from sklearn.ensemble import AdaBoostClassifier

  clfAB1 = AdaBoostClassifier(n_estimators=100, random_state=0)

  

  clfAB_Basic = clfAB1.fit(x_Train, y_Train)
  y_pred_Basic = clfAB_Basic.predict(x_Test)
  list_AB_Pre_Basic.append(precision_score(y_Test, y_pred_Basic,average='weighted'))
  list_AB_f1_Basic.append(f1_score(y_Test, y_pred_Basic, average='weighted'))
  list_AB_Acc_Basic.append(accuracy_score(y_Test, y_pred_Basic))

  clfAB2 = AdaBoostClassifier(n_estimators=100, random_state=0)

  clfABpca2 = clfAB2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfABpca2.predict(x_pca2_data_test)
  list_AB_Pre_PCA2.append(precision_score(y_Test, y_predPca2, average='weighted'))
  list_AB_f1_PCA2.append(f1_score(y_Test, y_predPca2, average='weighted'))
  list_AB_Acc_PCA2.append(accuracy_score(y_Test, y_predPca2))

  clfAB3 = AdaBoostClassifier(n_estimators=100, random_state=0)

  clfABpca10 = clfAB3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfABpca10.predict(x_pca10_data_test)
  list_AB_Pre_PCA10.append(precision_score(y_Test, y_predPca10, average='weighted'))
  list_AB_f1_PCA10.append(f1_score(y_Test, y_predPca10, average='weighted'))
  list_AB_Acc_PCA10.append(accuracy_score(y_Test, y_predPca10))


  clfAB4 = AdaBoostClassifier(n_estimators=100, random_state=0)

  clfAB_CMF = clfAB4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfAB_CMF.predict(x_Test_corr)
  list_AB_Pre_CM.append(precision_score(y_Test, y_pred_CMF, average='weighted'))
  list_AB_f1_CM.append(f1_score(y_Test, y_pred_CMF, average='weighted'))
  list_AB_Acc_CM.append(accuracy_score(y_Test, y_pred_CMF))

  clfAB5 = AdaBoostClassifier(n_estimators=100, random_state=0)

  # clfAB_CMFL = clfAB5.fit(new_x_train_corr, y_Train)
  # y_pred_CMFL = clfAB_CMFL.predict(new_x_test_corr)
  # list_AB_Pre_CMFL.append(precision_score(y_Test, y_pred_CMFL, average='weighted'))
  # list_AB_f1_CMFL.append(f1_score(y_Test, y_pred_CMFL, average='weighted'))
  # list_AB_Acc_CMFL.append(accuracy_score(y_Test, y_pred_CMFL))


  clfAB6 = AdaBoostClassifier(n_estimators=100, random_state=0)


  clfAB_selectK = clfAB6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfAB_selectK.predict(features_df_new_test)
  list_AB_Pre_SK.append(precision_score(y_Test, y_pred_selectK, average='weighted'))
  list_AB_f1_SK.append(f1_score(y_Test, y_pred_selectK, average='weighted'))
  list_AB_Acc_SK.append(accuracy_score(y_Test, y_pred_selectK))



  
  #4. XGBoostClassifier
  import xgboost as xg

  xgb_r1 = xg.XGBClassifier(max_depth=20)

  

  clfXGB_Basic = xgb_r1.fit(x_Train, y_Train)
  y_pred_Basic = clfXGB_Basic.predict(x_Test)
  list_XG_Pre_Basic.append(precision_score(y_Test, y_pred_Basic,average='weighted'))
  list_XG_f1_Basic.append(f1_score(y_Test, y_pred_Basic, average='weighted'))
  list_XG_Acc_Basic.append(accuracy_score(y_Test, y_pred_Basic))

  xgb_r2 = xg.XGBClassifier(max_depth=20)

  clfXGBpca2 = xgb_r2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfXGBpca2.predict(x_pca2_data_test)
  list_XG_Pre_PCA2.append(precision_score(y_Test, y_predPca2, average='weighted'))
  list_XG_f1_PCA2.append(f1_score(y_Test, y_predPca2, average='weighted'))
  list_XG_Acc_PCA2.append(accuracy_score(y_Test, y_predPca2))

  xgb_r3 = xg.XGBClassifier(max_depth=20)


  clfXGBpca10 = xgb_r3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfXGBpca10.predict(x_pca10_data_test)
  list_XG_Pre_PCA10.append(precision_score(y_Test, y_predPca10, average='weighted'))
  list_XG_f1_PCA10.append(f1_score(y_Test, y_predPca10, average='weighted'))
  list_XG_Acc_PCA10.append(accuracy_score(y_Test, y_predPca10))


  xgb_r4 = xg.XGBClassifier(max_depth=20)

  x_Train_corr = x_Train_corr.to_numpy()
  x_Test_corr = x_Test_corr.to_numpy()

  clfXGB_CMF = xgb_r4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfXGB_CMF.predict(x_Test_corr)
  list_XG_Pre_CM.append(precision_score(y_Test, y_pred_CMF, average='weighted'))
  list_XG_f1_CM.append(f1_score(y_Test, y_pred_CMF, average='weighted'))
  list_XG_Acc_CM.append(accuracy_score(y_Test, y_pred_CMF))

  xgb_r5 = xg.XGBClassifier(max_depth=20)

  # clfXGB_CMFL = xgb_r5.fit(new_x_train_corr, y_Train)
  # y_pred_CMFL = clfXGB_CMFL.predict(new_x_test_corr)
  # list_XG_Pre_CMFL.append(precision_score(y_Test, y_pred_CMFL, average='weighted'))
  # list_XG_f1_CMFL.append(f1_score(y_Test, y_pred_CMFL, average='weighted'))
  # list_XG_Acc_CMFL.append(accuracy_score(y_Test, y_pred_CMFL))


  xgb_r6 = xg.XGBClassifier(max_depth=20)

  features_df_new_train = features_df_new_train.to_numpy()
  features_df_new_test = features_df_new_test.to_numpy()
  clfXGB_selectK = xgb_r6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfXGB_selectK.predict(features_df_new_test)
  list_XG_Pre_SK.append(precision_score(y_Test, y_pred_selectK, average='weighted'))
  list_XG_f1_SK.append(f1_score(y_Test, y_pred_selectK, average='weighted'))
  list_XG_Acc_SK.append(accuracy_score(y_Test, y_pred_selectK))

  print("done")


done
done
done
done
done


In [ ]:
from statistics import mean
  
def Average(lst):
  return mean(lst)

In [ ]:
print("Logistic Regression :")
print("----------------------")
print("Basic ==>                          Precision: ",round(Average(list_LR_Pre_Basic), 4),"  F1: ",round(Average(list_LR_f1_Basic), 4)," Accuracy: ",round(Average(list_LR_Acc_Basic), 4))
print()
print("PCA with n = 2 ==>                 Precision: ",round(Average(list_LR_Pre_PCA2), 4),"  F1: ",round(Average(list_LR_f1_PCA2), 4)," Accuracy: ",round(Average(list_LR_Acc_PCA2), 4))
print()
print("PCA with n = 10 ==>                Precision: ",round(Average(list_LR_Pre_PCA10), 4),"  F1: ",round(Average(list_LR_f1_PCA10), 4)," Accuracy: ",round(Average(list_LR_Acc_PCA10), 4))
print()
print("Correlation Matrix on Features ==> Precision: ",round(Average(list_LR_Pre_CM), 4),"  F1: ",round(Average(list_LR_f1_CM), 4)," Accuracy: ",round(Average(list_LR_Acc_CM), 4))
print()
print("SelectK best with k = 200 ==>      Precision: ",round(Average(list_LR_Pre_SK), 4),"  F1: ",round(Average(list_LR_f1_SK), 4)," Accuracy: ",round(Average(list_LR_Acc_SK), 4))
print()
print()

print("Radom Forest :")
print("----------------------")
print("Basic ==>                          Precision: ",round(Average(list_RF_Pre_Basic), 4),"  F1: ",round(Average(list_RF_f1_Basic), 4)," Accuracy: ",round(Average(list_RF_Acc_Basic), 4))
print()
print("PCA with n = 2 ==>                 Precision: ",round(Average(list_RF_Pre_PCA2), 4),"  F1: ",round(Average(list_RF_f1_PCA2), 4)," Accuracy: ",round(Average(list_RF_Acc_PCA2), 4))
print()
print("PCA with n = 10 ==>                Precision: ",round(Average(list_RF_Pre_PCA10), 4),"  F1: ",round(Average(list_RF_f1_PCA10), 4)," Accuracy: ",round(Average(list_RF_Acc_PCA10), 4))
print()
print("Correlation Matrix on Features ==> Precision: ",round(Average(list_RF_Pre_CM), 4),"  F1: ",round(Average(list_RF_f1_CM), 4)," Accuracy: ",round(Average(list_RF_Acc_CM), 4))
print()
print("SelectK best with k = 200 ==>      Precision: ",round(Average(list_RF_Pre_SK), 4),"  F1: ",round(Average(list_RF_f1_SK), 4)," Accuracy: ",round(Average(list_RF_Acc_SK), 4))
print()
print()

print("Adaboost :")
print("----------------------")
print("Basic ==>                          Precision: ",round(Average(list_AB_Pre_Basic), 4),"  F1: ",round(Average(list_AB_f1_Basic), 4)," Accuracy: ",round(Average(list_AB_Acc_Basic), 4))
print()
print("PCA with n = 2 ==>                 Precision: ",round(Average(list_AB_Pre_PCA2), 4),"  F1: ",round(Average(list_AB_f1_PCA2), 4)," Accuracy: ",round(Average(list_AB_Acc_PCA2), 4))
print()
print("PCA with n = 10 ==>                Precision: ",round(Average(list_AB_Pre_PCA10), 4),"  F1: ",round(Average(list_AB_f1_PCA10), 4)," Accuracy: ",round(Average(list_AB_Acc_PCA10), 4))
print()
print("Correlation Matrix on Features ==> Precision: ",round(Average(list_AB_Pre_CM), 4),"  F1: ",round(Average(list_AB_f1_CM), 4)," Accuracy: ",round(Average(list_AB_Acc_CM), 4))
print()
print("SelectK best with k = 200 ==>      Precision: ",round(Average(list_AB_Pre_SK), 4),"  F1: ",round(Average(list_AB_f1_SK), 4)," Accuracy: ",round(Average(list_AB_Acc_SK), 4))
print()
print()

print("XgBoost :")
print("----------------------")
print("Basic ==>                          Precision: ",round(Average(list_XG_Pre_Basic), 4),"  F1: ",round(Average(list_XG_f1_Basic), 4)," Accuracy: ",round(Average(list_XG_Acc_Basic), 4))
print()
print("PCA with n = 2 ==>                 Precision: ",round(Average(list_XG_Pre_PCA2), 4),"  F1: ",round(Average(list_XG_f1_PCA2), 4)," Accuracy: ",round(Average(list_XG_Acc_PCA2), 4))
print()
print("PCA with n = 10 ==>                Precision: ",round(Average(list_XG_Pre_PCA10), 4),"  F1: ",round(Average(list_XG_f1_PCA10), 4)," Accuracy: ",round(Average(list_XG_Acc_PCA10), 4))
print()
print("Correlation Matrix on Features ==> Precision: ",round(Average(list_XG_Pre_CM), 4),"  F1: ",round(Average(list_XG_f1_CM), 4)," Accuracy: ",round(Average(list_XG_Acc_CM), 4))
print()
print("SelectK best with k = 200 ==>      Precision: ",round(Average(list_XG_Pre_SK), 4),"  F1: ",round(Average(list_XG_f1_SK), 4)," Accuracy: ",round(Average(list_XG_Acc_SK), 4))
print()
print()



Logistic Regression :
----------------------
Basic ==>                          Precision:  0.3247   F1:  0.3244  Accuracy:  0.3961

PCA with n = 2 ==>                 Precision:  0.4372   F1:  0.1931  Accuracy:  0.2686

PCA with n = 10 ==>                Precision:  0.421   F1:  0.194  Accuracy:  0.2682

Correlation Matrix on Features ==> Precision:  0.3247   F1:  0.3244  Accuracy:  0.3961

SelectK best with k = 200 ==>      Precision:  0.1713   F1:  0.2399  Accuracy:  0.4025


Radom Forest :
----------------------
Basic ==>                          Precision:  0.7508   F1:  0.7442  Accuracy:  0.753

PCA with n = 2 ==>                 Precision:  0.3482   F1:  0.2046  Accuracy:  0.3055

PCA with n = 10 ==>                Precision:  0.4107   F1:  0.3397  Accuracy:  0.4493

Correlation Matrix on Features ==> Precision:  0.7508   F1:  0.7442  Accuracy:  0.753

SelectK best with k = 200 ==>      Precision:  0.7513   F1:  0.7473  Accuracy:  0.755


Adaboost :
----------------------
Basic 

**Padel**

In [ ]:
chemTasteDB = pd.read_csv("/content/drive/MyDrive/Capstone/ChemTasteDB + Sweet_Bitter/chemTasteDBPadelMerged.csv")

In [ ]:
chemTasteDB = chemTasteDB.dropna()

In [ ]:
chemTasteDB.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5698 entries, 0 to 5727
Columns: 1443 entries, Class Taste to Zagreb
dtypes: float64(1309), int64(133), object(1)
memory usage: 62.8+ MB


In [ ]:
Y = chemTasteDB['Class Taste']
X = chemTasteDB.drop(['Class Taste'],axis=1)

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5698 entries, 0 to 5727
Columns: 1442 entries, nAcid to Zagreb
dtypes: float64(1309), int64(133)
memory usage: 62.7 MB


In [ ]:
Y=Y.to_frame()

In [ ]:
Y

,Class Taste
0,Umaminess
1,Sweetness
2,Bitterness
3,Sweetness
4,Bitterness
...,...
5723,Sweetness
5724,Sweetness
5725,Sweetness
5726,Bitterness


In [ ]:
x = X.to_numpy()
y = Y.to_numpy()

In [ ]:
import numpy as np
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)

list_LR_Pre_Basic = []
list_LR_f1_Basic = []
list_LR_Acc_Basic = []

list_LR_Pre_PCA2 = []
list_LR_f1_PCA2 = []
list_LR_Acc_PCA2 = []

list_LR_Pre_PCA10 = []
list_LR_f1_PCA10 = []
list_LR_Acc_PCA10 = []

list_LR_Pre_CM = []
list_LR_f1_CM = []
list_LR_Acc_CM = []

list_LR_Pre_CMFL = []
list_LR_f1_CMFL = []
list_LR_Acc_CMFL = []

list_LR_Pre_SK = []
list_LR_f1_SK = []
list_LR_Acc_SK = []

#==============================

list_RF_Pre_Basic = []
list_RF_f1_Basic = []
list_RF_Acc_Basic = []

list_RF_Pre_PCA2 = []
list_RF_f1_PCA2 = []
list_RF_Acc_PCA2 = []

list_RF_Pre_PCA10 = []
list_RF_f1_PCA10 = []
list_RF_Acc_PCA10 = []

list_RF_Pre_CM = []
list_RF_f1_CM = []
list_RF_Acc_CM = []

list_RF_Pre_CMFL = []
list_RF_f1_CMFL = []
list_RF_Acc_CMFL = []

list_RF_Pre_SK = []
list_RF_f1_SK = []
list_RF_Acc_SK = []


#==============================

list_AB_Pre_Basic = []
list_AB_f1_Basic = []
list_AB_Acc_Basic = []

list_AB_Pre_PCA2 = []
list_AB_f1_PCA2 = []
list_AB_Acc_PCA2 = []

list_AB_Pre_PCA10 = []
list_AB_f1_PCA10 = []
list_AB_Acc_PCA10 = []

list_AB_Pre_CM = []
list_AB_f1_CM = []
list_AB_Acc_CM = []

list_AB_Pre_CMFL = []
list_AB_f1_CMFL = []
list_AB_Acc_CMFL = []

list_AB_Pre_SK = []
list_AB_f1_SK = []
list_AB_Acc_SK = []


#==============================

list_XG_Pre_Basic = []
list_XG_f1_Basic = []
list_XG_Acc_Basic = []

list_XG_Pre_PCA2 = []
list_XG_f1_PCA2 = []
list_XG_Acc_PCA2 = []

list_XG_Pre_PCA10 = []
list_XG_f1_PCA10 = []
list_XG_Acc_PCA10 = []

list_XG_Pre_CM = []
list_XG_f1_CM = []
list_XG_Acc_CM = []

list_XG_Pre_CMFL = []
list_XG_f1_CMFL = []
list_XG_Acc_CMFL = []

list_XG_Pre_SK = []
list_XG_f1_SK = []
list_XG_Acc_SK = []


for train_index, test_index in kf.split(x):
  x_Train, x_Test = x[train_index], x[test_index]
  y_Train, y_Test = y[train_index], y[test_index]


  #Preparing data for PCA
  pca2 = PCA(n_components=2)
  x_pca2_data_train = pca2.fit_transform(x_Train)
  x_pca2_data_test = pca2.fit_transform(x_Test)

  pca10 = PCA(n_components=10)
  x_pca10_data_train = pca10.fit_transform(x_Train)
  x_pca10_data_test = pca10.fit_transform(x_Test)

  
  #Preparing data for Correlation matrix on features
  x_Train = pd.DataFrame(x_Train,columns = X.columns)
  y_Train = pd.DataFrame(y_Train,columns = Y.columns)
  x_Test = pd.DataFrame(x_Test,columns = X.columns)
  y_Test = pd.DataFrame(y_Test,columns = Y.columns)

  train_data = pd.concat([x_Train,y_Train],axis=1)
  test_data = pd.concat([x_Test,y_Test],axis=1)
  train_corr , test_corr = correlation_check(train_data, test_data,0.8)


  x_Train_corr = train_corr.drop(['Class Taste'],axis=1)
  x_Test_corr = test_corr.drop(['Class Taste'],axis=1)

  
  
  #Preparing data for Correlation matrix on features and Labels

  a = x_Train_corr.corr(method ='pearson').abs()
  df_temp = a.head(1)
  df_temp.fillna(0,inplace = True)

  colList = []
  for i in df_temp.columns:
    if(df_temp[i].item() < 0.2):
      colList.append(i)

  for i in colList:
    new_x_train_corr = x_Train_corr.drop(i,axis = 1)
    new_x_test_corr = x_Test_corr.drop(i,axis = 1)
  

  #Preparing data for selectkBest
  from sklearn.feature_selection import SelectKBest, f_classif
  
  X_new = SelectKBest(f_classif, k=200).fit(x_Train, y_Train)
  cols = X_new.get_support(indices=True)
  features_df_new_train = x_Train.iloc[:,cols]
  features_df_new_test = x_Test.iloc[:,cols]


  x_Train = x_Train.to_numpy()
  x_Test = x_Test.to_numpy()
  y_Train = y_Train.to_numpy()
  y_Test = y_Test.to_numpy()

  # Logistic Regression
  from sklearn.linear_model import LogisticRegression

  clfLR1 = LogisticRegression(random_state=0)

  clfLR_Basic = clfLR1.fit(x_Train, y_Train)
  y_pred_Basic = clfLR_Basic.predict(x_Test)

  list_LR_Pre_Basic.append(precision_score(y_Test, y_pred_Basic,average='weighted'))
  list_LR_f1_Basic.append(f1_score(y_Test, y_pred_Basic, average='weighted'))
  list_LR_Acc_Basic.append(accuracy_score(y_Test, y_pred_Basic))
  
  clfLR2 = LogisticRegression(random_state=0)

  clfLRpca2 = clfLR2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfLRpca2.predict(x_pca2_data_test)

  list_LR_Pre_PCA2.append(precision_score(y_Test, y_predPca2, average='weighted'))
  list_LR_f1_PCA2.append(f1_score(y_Test, y_predPca2, average='weighted'))
  list_LR_Acc_PCA2.append(accuracy_score(y_Test, y_predPca2))


  clfLR3 = LogisticRegression(random_state=0)

  clfLRpca10 = clfLR3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfLRpca10.predict(x_pca10_data_test)
 
  list_LR_Pre_PCA10.append(precision_score(y_Test, y_predPca10, average='weighted'))
  list_LR_f1_PCA10.append(f1_score(y_Test, y_predPca10, average='weighted'))
  list_LR_Acc_PCA10.append(accuracy_score(y_Test, y_predPca10))


  clfLR4 = LogisticRegression(random_state=0)

  clfLR_CMF = clfLR4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfLR_CMF.predict(x_Test_corr)
  
  list_LR_Pre_CM.append(precision_score(y_Test, y_pred_CMF, average='weighted'))
  list_LR_f1_CM.append(f1_score(y_Test, y_pred_CMF, average='weighted'))
  list_LR_Acc_CM.append(accuracy_score(y_Test, y_pred_CMF))

  clfLR5 = LogisticRegression(random_state=0)

  # clfLR_CMFL = clfLR5.fit(new_x_train_corr, y_Train)
  # y_pred_CMFL = clfLR_CMFL.predict(new_x_test_corr)
  
  # list_LR_Pre_CMFL.append(precision_score(y_Test, y_pred_CMFL, average='weighted'))
  # list_LR_f1_CMFL.append(f1_score(y_Test, y_pred_CMFL, average='weighted'))
  # list_LR_Acc_CMFL.append(accuracy_score(y_Test, y_pred_CMFL))

  clfLR6 = LogisticRegression(random_state=0)

  clfLR_selectK = clfLR6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfLR_selectK.predict(features_df_new_test)

  list_LR_Pre_SK.append(precision_score(y_Test, y_pred_selectK, average='weighted'))
  list_LR_f1_SK.append(f1_score(y_Test, y_pred_selectK, average='weighted'))
  list_LR_Acc_SK.append(accuracy_score(y_Test, y_pred_selectK))

  #=====================================================================================================================================================

  #2. Random forgest :
  from sklearn.ensemble import RandomForestClassifier

  clfRF1 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=25, criterion='entropy', random_state=30)
  
  

  clfRF_Basic = clfRF1.fit(x_Train, y_Train)
  y_pred_Basic = clfRF_Basic.predict(x_Test)
  list_RF_Pre_Basic.append(precision_score(y_Test, y_pred_Basic,average='weighted'))
  list_RF_f1_Basic.append(f1_score(y_Test, y_pred_Basic, average='weighted'))
  list_RF_Acc_Basic.append(accuracy_score(y_Test, y_pred_Basic))

  clfRF2 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=25, criterion='entropy', random_state=30)


  clfRFpca2 = clfRF2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfRFpca2.predict(x_pca2_data_test)
  list_RF_Pre_PCA2.append(precision_score(y_Test, y_predPca2, average='weighted'))
  list_RF_f1_PCA2.append(f1_score(y_Test, y_predPca2, average='weighted'))
  list_RF_Acc_PCA2.append(accuracy_score(y_Test, y_predPca2))

  clfRF3 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=25, criterion='entropy', random_state=30)


  clfRFpca10 = clfRF3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfRFpca10.predict(x_pca10_data_test)
  list_RF_Pre_PCA10.append(precision_score(y_Test, y_predPca10, average='weighted'))
  list_RF_f1_PCA10.append(f1_score(y_Test, y_predPca10, average='weighted'))
  list_RF_Acc_PCA10.append(accuracy_score(y_Test, y_predPca10))


  clfRF4 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=25, criterion='entropy', random_state=30)

  clfRF_CMF = clfRF4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfRF_CMF.predict(x_Test_corr)
  list_RF_Pre_CM.append(precision_score(y_Test, y_pred_CMF, average='weighted'))
  list_RF_f1_CM.append(f1_score(y_Test, y_pred_CMF, average='weighted'))
  list_RF_Acc_CM.append(accuracy_score(y_Test, y_pred_CMF))

  clfRF5 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=25, criterion='entropy', random_state=30)

  # clfLR_CMFL = clfRF5.fit(new_x_train_corr, y_Train)
  # y_pred_CMFL = clfLR_CMFL.predict(new_x_test_corr)
  # list_RF_Pre_CMFL.append(precision_score(y_Test, y_pred_CMFL, average='weighted'))
  # list_RF_f1_CMFL.append(f1_score(y_Test, y_pred_CMFL, average='weighted'))
  # list_RF_Acc_CMFL.append(accuracy_score(y_Test, y_pred_CMFL))

  clfRF6 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=25, criterion='entropy', random_state=30)

  clfRF_selectK = clfRF6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfRF_selectK.predict(features_df_new_test)
  list_RF_Pre_SK.append(precision_score(y_Test, y_pred_selectK, average='weighted'))
  list_RF_f1_SK.append(f1_score(y_Test, y_pred_selectK, average='weighted'))
  list_RF_Acc_SK.append(accuracy_score(y_Test, y_pred_selectK))



  


  #3. Adaboost
  from sklearn.ensemble import AdaBoostClassifier

  clfAB1 = AdaBoostClassifier(n_estimators=100, random_state=0)

  

  clfAB_Basic = clfAB1.fit(x_Train, y_Train)
  y_pred_Basic = clfAB_Basic.predict(x_Test)
  list_AB_Pre_Basic.append(precision_score(y_Test, y_pred_Basic,average='weighted'))
  list_AB_f1_Basic.append(f1_score(y_Test, y_pred_Basic, average='weighted'))
  list_AB_Acc_Basic.append(accuracy_score(y_Test, y_pred_Basic))

  clfAB2 = AdaBoostClassifier(n_estimators=100, random_state=0)

  clfABpca2 = clfAB2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfABpca2.predict(x_pca2_data_test)
  list_AB_Pre_PCA2.append(precision_score(y_Test, y_predPca2, average='weighted'))
  list_AB_f1_PCA2.append(f1_score(y_Test, y_predPca2, average='weighted'))
  list_AB_Acc_PCA2.append(accuracy_score(y_Test, y_predPca2))

  clfAB3 = AdaBoostClassifier(n_estimators=100, random_state=0)

  clfABpca10 = clfAB3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfABpca10.predict(x_pca10_data_test)
  list_AB_Pre_PCA10.append(precision_score(y_Test, y_predPca10, average='weighted'))
  list_AB_f1_PCA10.append(f1_score(y_Test, y_predPca10, average='weighted'))
  list_AB_Acc_PCA10.append(accuracy_score(y_Test, y_predPca10))


  clfAB4 = AdaBoostClassifier(n_estimators=100, random_state=0)

  clfAB_CMF = clfAB4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfAB_CMF.predict(x_Test_corr)
  list_AB_Pre_CM.append(precision_score(y_Test, y_pred_CMF, average='weighted'))
  list_AB_f1_CM.append(f1_score(y_Test, y_pred_CMF, average='weighted'))
  list_AB_Acc_CM.append(accuracy_score(y_Test, y_pred_CMF))

  clfAB5 = AdaBoostClassifier(n_estimators=100, random_state=0)

  # clfAB_CMFL = clfAB5.fit(new_x_train_corr, y_Train)
  # y_pred_CMFL = clfAB_CMFL.predict(new_x_test_corr)
  # list_AB_Pre_CMFL.append(precision_score(y_Test, y_pred_CMFL, average='weighted'))
  # list_AB_f1_CMFL.append(f1_score(y_Test, y_pred_CMFL, average='weighted'))
  # list_AB_Acc_CMFL.append(accuracy_score(y_Test, y_pred_CMFL))


  clfAB6 = AdaBoostClassifier(n_estimators=100, random_state=0)


  clfAB_selectK = clfAB6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfAB_selectK.predict(features_df_new_test)
  list_AB_Pre_SK.append(precision_score(y_Test, y_pred_selectK, average='weighted'))
  list_AB_f1_SK.append(f1_score(y_Test, y_pred_selectK, average='weighted'))
  list_AB_Acc_SK.append(accuracy_score(y_Test, y_pred_selectK))



  
  #4. XGBoostClassifier
  import xgboost as xg

  xgb_r1 = xg.XGBClassifier(max_depth=20)

  

  clfXGB_Basic = xgb_r1.fit(x_Train, y_Train)
  y_pred_Basic = clfXGB_Basic.predict(x_Test)
  list_XG_Pre_Basic.append(precision_score(y_Test, y_pred_Basic,average='weighted'))
  list_XG_f1_Basic.append(f1_score(y_Test, y_pred_Basic, average='weighted'))
  list_XG_Acc_Basic.append(accuracy_score(y_Test, y_pred_Basic))

  xgb_r2 = xg.XGBClassifier(max_depth=20)

  clfXGBpca2 = xgb_r2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfXGBpca2.predict(x_pca2_data_test)
  list_XG_Pre_PCA2.append(precision_score(y_Test, y_predPca2, average='weighted'))
  list_XG_f1_PCA2.append(f1_score(y_Test, y_predPca2, average='weighted'))
  list_XG_Acc_PCA2.append(accuracy_score(y_Test, y_predPca2))

  xgb_r3 = xg.XGBClassifier(max_depth=20)


  clfXGBpca10 = xgb_r3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfXGBpca10.predict(x_pca10_data_test)
  list_XG_Pre_PCA10.append(precision_score(y_Test, y_predPca10, average='weighted'))
  list_XG_f1_PCA10.append(f1_score(y_Test, y_predPca10, average='weighted'))
  list_XG_Acc_PCA10.append(accuracy_score(y_Test, y_predPca10))


  xgb_r4 = xg.XGBClassifier(max_depth=20)

  x_Train_corr = x_Train_corr.to_numpy()
  x_Test_corr = x_Test_corr.to_numpy()

  clfXGB_CMF = xgb_r4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfXGB_CMF.predict(x_Test_corr)
  list_XG_Pre_CM.append(precision_score(y_Test, y_pred_CMF, average='weighted'))
  list_XG_f1_CM.append(f1_score(y_Test, y_pred_CMF, average='weighted'))
  list_XG_Acc_CM.append(accuracy_score(y_Test, y_pred_CMF))

  xgb_r5 = xg.XGBClassifier(max_depth=20)

  # clfXGB_CMFL = xgb_r5.fit(new_x_train_corr, y_Train)
  # y_pred_CMFL = clfXGB_CMFL.predict(new_x_test_corr)
  # list_XG_Pre_CMFL.append(precision_score(y_Test, y_pred_CMFL, average='weighted'))
  # list_XG_f1_CMFL.append(f1_score(y_Test, y_pred_CMFL, average='weighted'))
  # list_XG_Acc_CMFL.append(accuracy_score(y_Test, y_pred_CMFL))


  xgb_r6 = xg.XGBClassifier(max_depth=20)

  features_df_new_train = features_df_new_train.to_numpy()
  features_df_new_test = features_df_new_test.to_numpy()
  clfXGB_selectK = xgb_r6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfXGB_selectK.predict(features_df_new_test)
  list_XG_Pre_SK.append(precision_score(y_Test, y_pred_selectK, average='weighted'))
  list_XG_f1_SK.append(f1_score(y_Test, y_pred_selectK, average='weighted'))
  list_XG_Acc_SK.append(accuracy_score(y_Test, y_pred_selectK))

  print("done")


done
done
done
done
done


In [ ]:
print("Logistic Regression :")
print("----------------------")
print("Basic ==>                          Precision: ",round(Average(list_LR_Pre_Basic), 4),"  F1: ",round(Average(list_LR_f1_Basic), 4)," Accuracy: ",round(Average(list_LR_Acc_Basic), 4))
print()
print("PCA with n = 2 ==>                 Precision: ",round(Average(list_LR_Pre_PCA2), 4),"  F1: ",round(Average(list_LR_f1_PCA2), 4)," Accuracy: ",round(Average(list_LR_Acc_PCA2), 4))
print()
print("PCA with n = 10 ==>                Precision: ",round(Average(list_LR_Pre_PCA10), 4),"  F1: ",round(Average(list_LR_f1_PCA10), 4)," Accuracy: ",round(Average(list_LR_Acc_PCA10), 4))
print()
print("Correlation Matrix on Features ==> Precision: ",round(Average(list_LR_Pre_CM), 4),"  F1: ",round(Average(list_LR_f1_CM), 4)," Accuracy: ",round(Average(list_LR_Acc_CM), 4))
print()
print("SelectK best with k = 200 ==>      Precision: ",round(Average(list_LR_Pre_SK), 4),"  F1: ",round(Average(list_LR_f1_SK), 4)," Accuracy: ",round(Average(list_LR_Acc_SK), 4))
print()
print()

print("Radom Forest :")
print("----------------------")
print("Basic ==>                          Precision: ",round(Average(list_RF_Pre_Basic), 4),"  F1: ",round(Average(list_RF_f1_Basic), 4)," Accuracy: ",round(Average(list_RF_Acc_Basic), 4))
print()
print("PCA with n = 2 ==>                 Precision: ",round(Average(list_RF_Pre_PCA2), 4),"  F1: ",round(Average(list_RF_f1_PCA2), 4)," Accuracy: ",round(Average(list_RF_Acc_PCA2), 4))
print()
print("PCA with n = 10 ==>                Precision: ",round(Average(list_RF_Pre_PCA10), 4),"  F1: ",round(Average(list_RF_f1_PCA10), 4)," Accuracy: ",round(Average(list_RF_Acc_PCA10), 4))
print()
print("Correlation Matrix on Features ==> Precision: ",round(Average(list_RF_Pre_CM), 4),"  F1: ",round(Average(list_RF_f1_CM), 4)," Accuracy: ",round(Average(list_RF_Acc_CM), 4))
print()
print("SelectK best with k = 200 ==>      Precision: ",round(Average(list_RF_Pre_SK), 4),"  F1: ",round(Average(list_RF_f1_SK), 4)," Accuracy: ",round(Average(list_RF_Acc_SK), 4))
print()
print()

print("Adaboost :")
print("----------------------")
print("Basic ==>                          Precision: ",round(Average(list_AB_Pre_Basic), 4),"  F1: ",round(Average(list_AB_f1_Basic), 4)," Accuracy: ",round(Average(list_AB_Acc_Basic), 4))
print()
print("PCA with n = 2 ==>                 Precision: ",round(Average(list_AB_Pre_PCA2), 4),"  F1: ",round(Average(list_AB_f1_PCA2), 4)," Accuracy: ",round(Average(list_AB_Acc_PCA2), 4))
print()
print("PCA with n = 10 ==>                Precision: ",round(Average(list_AB_Pre_PCA10), 4),"  F1: ",round(Average(list_AB_f1_PCA10), 4)," Accuracy: ",round(Average(list_AB_Acc_PCA10), 4))
print()
print("Correlation Matrix on Features ==> Precision: ",round(Average(list_AB_Pre_CM), 4),"  F1: ",round(Average(list_AB_f1_CM), 4)," Accuracy: ",round(Average(list_AB_Acc_CM), 4))
print()
print("SelectK best with k = 200 ==>      Precision: ",round(Average(list_AB_Pre_SK), 4),"  F1: ",round(Average(list_AB_f1_SK), 4)," Accuracy: ",round(Average(list_AB_Acc_SK), 4))
print()
print()

print("XgBoost :")
print("----------------------")
print("Basic ==>                          Precision: ",round(Average(list_XG_Pre_Basic), 4),"  F1: ",round(Average(list_XG_f1_Basic), 4)," Accuracy: ",round(Average(list_XG_Acc_Basic), 4))
print()
print("PCA with n = 2 ==>                 Precision: ",round(Average(list_XG_Pre_PCA2), 4),"  F1: ",round(Average(list_XG_f1_PCA2), 4)," Accuracy: ",round(Average(list_XG_Acc_PCA2), 4))
print()
print("PCA with n = 10 ==>                Precision: ",round(Average(list_XG_Pre_PCA10), 4),"  F1: ",round(Average(list_XG_f1_PCA10), 4)," Accuracy: ",round(Average(list_XG_Acc_PCA10), 4))
print()
print("Correlation Matrix on Features ==> Precision: ",round(Average(list_XG_Pre_CM), 4),"  F1: ",round(Average(list_XG_f1_CM), 4)," Accuracy: ",round(Average(list_XG_Acc_CM), 4))
print()
print("SelectK best with k = 200 ==>      Precision: ",round(Average(list_XG_Pre_SK), 4),"  F1: ",round(Average(list_XG_f1_SK), 4)," Accuracy: ",round(Average(list_XG_Acc_SK), 4))
print()
print()



Logistic Regression :
----------------------
Basic ==>                          Precision:  0.4859   F1:  0.472  Accuracy:  0.5265

PCA with n = 2 ==>                 Precision:  0.3844   F1:  0.2939  Accuracy:  0.4314

PCA with n = 10 ==>                Precision:  0.4194   F1:  0.2879  Accuracy:  0.4194

Correlation Matrix on Features ==> Precision:  0.4471   F1:  0.4034  Accuracy:  0.476

SelectK best with k = 200 ==>      Precision:  0.5928   F1:  0.585  Accuracy:  0.6174


Radom Forest :
----------------------
Basic ==>                          Precision:  0.8238   F1:  0.8198  Accuracy:  0.8296

PCA with n = 2 ==>                 Precision:  0.2324   F1:  0.1598  Accuracy:  0.2759

PCA with n = 10 ==>                Precision:  0.35   F1:  0.3044  Accuracy:  0.3987

Correlation Matrix on Features ==> Precision:  0.8255   F1:  0.8211  Accuracy:  0.8322

SelectK best with k = 200 ==>      Precision:  0.8241   F1:  0.8208  Accuracy:  0.8296


Adaboost :
----------------------
Basic 